In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from collections import Counter
from torch.utils.data import Dataset, DataLoader
from nltk.tokenize import word_tokenize
import nltk

In [2]:
document = """
Human 1: Hi!
Human 2: What is your favorite holiday?
Human 1: one where I get to meet lots of different people.
Human 2: What was the most number of people you have ever met during a holiday?
Human 1: Hard to keep a count. Maybe 25.
Human 2: Which holiday was that?
Human 1: I think it was Australia
Human 2: Do you still talk to the people you met?
Human 1: Not really. The interactions are usually short-lived but it's fascinating to learn where people are coming from and what matters to them
Human 2: Yea, me too. I feel like God often puts strangers in front of you, and gives you an opportunity to connect with them in that moment in deeply meaningful ways. Do you ever feel like you know things about strangers without them telling you?
Human 1: what do you mean?
Human 2: I think it's like a 6th sense, often seen as "cold readings" to people, but can be remarkably accurate. I once sat next to a man in a coffee and I felt a pain in my back. I asked the stranger if he had a pain. It turns out that he did in the exact spot, and said he pulled a muscle while dancing at a party. I had never met the man before and never saw him again.
Human 1: Wow! That's interesting, borderline spooky
Human 2: There's this practice called "Treasure Hunting" that's kind of a fun game you play in a public place. There's a book called "The Ultimate Treasure Hunt" that talks about it. You use your creativity to imagine people you will meet, and you write down a description, then you associate them with a positive message or encouraging word. Maybe you saw a teenage boy in a red hat at the shopping mall in your imagination, then while at the mall, you may find someone who matches that description. You show that you have a message for him and that you have a message for a boy in a red hat. You then give him a message of kindness or whatever was on your heart. You have no idea, sometimes you meet someone who is having a really hard day, and it brings them to tears to have a stranger show them love.
Human 1: So, do you do treasure hunting often?
Human 2: I did more when I was in grad school (and had more time). I would usually go with friends. For a while I would go to the farmers market in Santa Cruz every week and try to feel if there is something I am supposed to tell a stranger. Usually, they are vague hope-filled messages, but it's weird when I blurt out something oddly specific.
Human 1: Hi
Human 2: Any plans for the weekend?
Human 1: my friends are gonna visit me this weekend. we might go hiking!
Human 2: That's great! How's the weather over the weekend? I hope its warm.
Human 1: Should be very sunny! you?
Human 2: Cool! very depressing plans ... stay home and work 😞 I have a project deadline very close.
Human 1: 😐 hope you get your work done very soon! a bug free weekend!
Human 2: Right, very anxious! where do you plan to go for a hike?
Human 1: I am going to Diablo!
Human 2: Nice, where is that place? I haven't been there
Human 1: hours drive from here. still in bay area
Human 2: That's cool! How long is the hike?
Human 1:  Actually no idea, but it will take the entire day for that.
Human 2: nice! sounds fun!
Human 1: Hi!
Human 2: Hey there! What's up???
Human 1: Nothing much, how you doin?
Human 2: I'm in New York this week for Thanksgiving. I'm squatting in the office today and I caught up with an old friend of mine :D
Human 1: Oh wow! Sounds like fun! When was the last time you had seen this friend?
Human 2: The last time in New York, back in June.
Human 1: Ohh okay. I was going to say if it had been a long time maybe it'd be awkward...
Human 2: Haha, I guess if it's been a very long time there's almost too many life events to catch up on.. especially recently
Human 1: Oh really? Has a lot changed in your life recently?
Human 2: Haha it's probably too much to go into at the moment. Let's just say life is an exciting experience. How about you?
Human 1: Ahhh sounds exciting indeed! My life is pretty bland. I like routine, but sometimes I wish I had more time for adventures!
Human 2: What kinds of adventures?? Any ones that I would be able to join you on?
Human 1: Hmmmm. I really want to try bull riding. Do you have any interest in that?
Human 2: I'd love to try! Can we schedule something for next week?
Human 1: Sure! What does your Saturday look like?
Human 2: Saturday looks pretty good, shall we shoot for something in the morning?
Human 1: Hi!
Human 2: hey
Human 1: is it raining pretty bad today?
Human 2: yeah, can walk too far to see all the foodtruck options
Human 1: surprising that the rain started early this year... I don't like them too much. They make days gloomy
Human 2: yeah but I think it's good to have some rainy days in bay area, it's pretty dry here 😛
Human 1: Where I grew up, we had lots of water trouble too...
Human 2: yeah like wise, I've seen a pretty bad snowstorm when I was at my undergrad school, all flights canceled and traffics went down
Human 1: Haha... I don't think I can survive in that weather ever. Just the rains at 50 degrees make me want to sit in heated rroms
Human 2: yeah how do you like it in bay area though? I think we need more rain here
Human 1: people say there is drought here... but we have 24 hours water supply here ... lol... never seen that in a drought ridden area
Human 2: it is pretty dry in the mountains I believe, that's what causes fire
Human 1: hmm.... okay. Climate change talk this morning was pretty darn interesting. did you see it?
Human 2: nope, what does it say?
Human 1: they were talking about how AI is helping climate change. Nice use of upcoming tech.
Human 1: Hi.
Human 2: Helloooooo!
Human 1: How are you? How is your day?
Human 2: Good. Don't have much to do today, feels good. How are you?
Human 1: I'm dressed very wel today so I feel good! I've been reading a lot about the psychology of positive outlook.
Human 2: So what's your outlook? Something blue?
Human 1: Yes. Blue is a tranquil colour. It's a good metaphor. Do you have good advice for positivity?
Human 2: You should drink more water, do some push up, and sleep early.
Human 1: Hi!
Human 2: Hey, how are you?
Human 1: I'm a bit sad. I miss my cat.
Human 2: Oh no… Have you sent out the missing cat posters? Hope your cat is alright!
Human 1: Posters is a great idea. So far I've just tried banging her catfood dish and shouting her name. Anyway, how is your day going so far?
Human 2: Yea, I know they love the plastic bag sound all the time. I am good, nothing special though.
Human 1: If you could go anywhere on vacation, where would you go?
Human 2: I like rainforest, but I know it requires extensive training beforehand.
Human 1: I heard there are rainforests in southeast Asia where you can zipline from tree to tree.
Human 2: I am afraid I will be scared of doing this :)
Human 1: I won't lie, it sounds scary. I'm scared right now just thinking about it.
Human 2: I don't know if there is any medication for acrophobia. I want to take plenty of it if I really have to do it.
Human 1: If there isn't one, you should invent it, and then make millions
Human 2: That's a great idea! Maybe alcohol is such a thing.
Human 1: Ha! Don't drink and zipline, mate!
Human 2: Oops. I won't do it again. Ha
Human 1: Hi!
Human 2: Hey sup
Human 1: not much. any plans this weekend?
Human 2: I'm going to try that thing where you hang from a wire as you go down. do you know what is it called?
Human 1: ziplining?
Human 2: that's the one! have you ever tried it?
Human 1: i have a couple years ago. it's quite a unique experience
Human 2: where did you do it?
Human 1: i forgot where it was, it wasn't local i don't think though
Human 2: no worries. what's the most exciting thing you ever done?
Human 1: that's a hard question and i'm tired so i'm going to go. see you
Human 2: sure. are you just going home now?
Human 1: no, i'm going to get a massage first
Human 2: nice. what type?
Human 1: traditional kind
Human 2: yeah I want to get one too soon
Human 1: you should! it's relaxing after a long day. talk to you later!
Human 2: ttyl!
Human 1: Hi!
Human 2: Hello, have you seen any good movies lately?
Human 1: I watched a few lately, but nothing is as good as Avatar. what's your favorite?
Human 2: I have never seen Avatar, what is it about? I really enjoy the Avenger movies
Human 1: it's a science-fiction movie with beautiful landscape of an imaginary nature with non-human creatures. people figured out a way to join that nature through Avatar transformation. the movie ends with a meaningful story of how human behaviors, e.g., cutting trees, have affected nature
Human 2: That sounds really cool! I think that movie did really well when it was in the box office so it must be good!
Human 1: yea. what else do you like to do beside movies?
Human 2: I enjoy baking cookies. I am on a quest to bake the best chocolate chip cookie 🙂 What about you?
Human 1: I enjoy eating 🙂
Human 2: so definitely would like to try your best chocolate cookie
Human 1: I will have to bake some soon and let you know. What types of food do you like to eat?
Human 2: thanks! I generally love noodle soups like Pho or Ramen :)
Human 1: Noodle soup is delicious! Do you make homemade noodle soup or do you prefer to go out?
Human 2: I prefer to go out. I'm not a good cook haha
Human 1: Same! Even though I bake, I cannot cook
Human 2: seems like we share a thing in common, yay!
Human 1: Hi!
Human 2: Good afternoon!
Human 1: How has your week been?
Human 2: So far so good. It is holiday season. So just chilling
Human 1: I think I'm getting sick with a cold 😞 So you should chill on my behalf too cause I'm out the game for all of December.
Human 2: lol Sorry to hear that. Are you planning anything fun for December?
Human 1: Nothing exciting. I'll be posted up at home for the most part. I did a lot of travelling this year so my budget would have stopped me even if I wasn't sick.
Human 2: 😂
Human 1: Do you have big plans?
Human 2: Yes! I am going to Hawaii! This will be my first time visiting Hawaii. Really excited about it.
Human 1: I love Hawaii. It's a good place to be. I like going there cause it's humid so I never have to put on lotion.
Human 2: lol this is the first time I heard from a boy who cares about humidity and lotion. I cannot agree more.
Human 1: Brooooo!!! It's so important. When I got to California beaches I have to carry 3 litres of lotion for the whole day.
Human 2: 😂
Human 1: Hi!
Human 2: Oh hello. Long time no talk. How's the day going for yuo?
Human 1: Very well, thanks for asking. How has your day been?
Human 2: Getting better. I just recovered from a cold. I got wet in the rain last week. Are you planning anything for the holidays?
Human 1: Glad to hear you're better. Sorry to hear you were sick. I was sick a couple of weeks ago with a bad cough. There's definitely a bug going around. Admit I just want to stay healthy for the holidays and plan to relax.
Human 2: Oh same here. I think relaxing at home should be counted among the best ways to enjoy the holidays.
Human 1: Definitely! I know a lot of folks travel for the holidays, but I'm happy to stay home myself!
Human 2: I'm getting there. Every year until last year, I tried to go somewhere for the Christmas / New Year, and then I got bored traveling. lol not sure if that means I'm getting old?
Human 1: Me too. Now I have folks come visit me for the holidays! But that's also tiresome..
Human 2: Are you doing any home decorating then?
Human 1: Yes! We set up an eco-friendly (i.e. fake) Christmas tree and put up some colorful LED lights which is very festive.
Human 2: I think I'm copying you. Me and my wife plan to decorate and Christmas tree too. We bought most of the decorative stuffs from the stores, but haven't yet to buy the tree.
Human 1: Buying a tree is a neat experience. I was torn between buying an artificial/eco-friendly/fake one vs. a real one that smells like fresh pine. In the end, we opted for the one that we can disassemble every year.
Human 2: I see. Artificial anything is better, from tree to intelligence, huh?
Human 1: Oh, very clever pun! I like it! Depends. I remember having real Christmas trees from childhood, but these days with climate change, I think not chopping down a tree just to decorate it and then throw it out in a month is the more responsible thing to do.
Human 2: I see. It's probably also cheaper. I'll buy an artificial one too. Do you have any suggestions for the store?
Human 1: Admit my favorite store is Target, plus they often have good deals.
Human 2: Ah that's great. My wife also likes Target a lot. She even made a Target credit card because she comes to that store very often. Okay thanks for the suggestion. I'll check out Target.
Human 1: Great, I hope you find a nice tree.
Human 1: Hi!
Human 2: Hey
Human 1: How's your day going?
Human 2: pretty good. yours?
Human 1: Ehh it's fine. I didn't do so well on that history test, actually..
Human 2: oh what happened?
Human 1: Apparently Christopher Columbus didn't fight in the Civil War :')
Human 2: hahah wait for real?
Human 1: I know right! Are you taking History next semester?
Human 2: No I'm not in school anymore
Human 1: Oh I see. What do you do?
Human 2: I train and compete in horse vaulting
Human 1: Oh wow. Were you born a horse, or were you turned into one?
Human 2: lol you're too funny
Human 1: Just kidding. That sounds pretty cool! Is it your job?
Human 2: Yeah, but I part time work on a farm. Helping with a bit of everything
Human 1: Wow, sounds very busy! Do you with money at those horse vaulting competitions?
Human 2: Yeah some. enough to get by
Human 1: Hi!
Human 2: Hello
Human 1: Do you have a favourite flower?
Human 2: hmm, I haven't thought about that much, but i think lotus should be one of my favorites. Why do you ask?
Human 1: I'm working on a theory. Why does the lotus spring to mind?
Human 2: Nice! Lotus looks pretty cool and It has some delightful vibe. So what is this research about?
Human 1: Oh, it's not research! Just a personal theory. I think that flower preferences are more revealing of personality than people appreciate.
Human 2: Interesting! Whats your favorite flower?
Human 1: The gerbera. It's like a cartoon flower. As if you drew "flower" with a crayon and then it came to life.
Human 2: Nice, i would love know more about your theory. Like how you can deduce personality from flower preference.
Human 1: Ok, step 1 is, you ask someone what their favourite flower is. Pretty much like what we just did. Does that make sense so far?
Human 2: yes
Human 1: Cool. Step 2: talk with the person some more, and ask them some more questions, and gradually develop a sense of what they're like, over the course of maybe two to five years. And voila
Human 2: Hehe, i think you should publish this someday :)
Human 1: Why thank you, that's a wonderful idea!
Human 1: Hi!
Human 2: Hey how's it going
Human 1: It's good it's good. How are you?
Human 2: good. it's really hot today. I think I'm going to the pool
Human 1: Oh nice! Where do you live?
Human 2: I live in Tokyo, Japan
Human 1: Ahh yes, Japan is hot during the summer. Last time I was in Kyoto it was 114 degrees....
Human 2: oh have you been?
Human 1: Yes yes. I've been to Tokyo as well. It's so nice!
Human 2: what did you do here?
Human 1: Oh everything! I went to an onsen, the fish market, disney land and giant robot fighting show haha
Human 2: lol why did you come to Japan just to go to Disney land?
Human 1: The Disney lands are all different! There's also Disney Sea, which is completely unique!
Human 2: oh neat. I haven't heard about that robot fighting show. where is that??
Human 1: I don't really remember what part of town it was in. It was pretty cool though - I'm sure you can find it if you google "giant robot fighting show tokyo" haha
Human 2: lol ok
Human 1: Hi!
Human 2: Have you seen any good movies lately?
Human 1: Last weekend I saw "The Parasite." Ever heard of it?
Human 2: No. Why did you pick that movie?
Human 1: My friend wanted to see it. It has great reviews on IMDB and Rotten Tomatoes! What did you do last weekend?
Human 2: I played music and worked on some side projects. I also started watching the new Disney service.
Human 1: Oooo the Mandalorian?!?!
Human 2: Mostly, the deleted scenes from Avengers.. lol
Human 1: lol Are you a big Marvel fan?
Human 2: I loved the X-Men as a kid, and even collected the comic cards. Recently, I got very into the Marvel Cinematic Universe movies. How many Avengers movies have you seen?
Human 1: I've only seen Spiderman. Honestly it was a little too scary and so I don't think I can bring myself to watch the other Marvel movies! haha
Human 2: Oh!-- I have a friend who looks like the actor who plays Spiderman.
Human 1: Oh really? To be honest I think the actor is not that good looking, so not so surprising! haha
Human 2: Yea. I think Loki is the most handsome 😀
Human 1: Who is Loki? I've never heard that name before
Human 2: He's the adopted brother of Thor, God of thunder, and is burdened with glorious purpose. Do you feel that burden?
Human 1: Hi!
Human 2: Hey, what's up?
Human 1: Just chillin'. how are you?
Human 2: I'm pretty good, thanks.
Human 1: Do anything interesting today?
Human 2: I went to the local cafe and had a double espresso. It was delicious. What about you?
Human 1: Oh that's cool! I actually went to an amusement park and went on my first roller coaster!
Human 2: Oh my gosh. What was it like??
Human 1: It was scary! It was actually Kingda Ka, the world's tallest roller coaster. Ever heard of it?
Human 2: No, never heard of it. But I'm not really a coaster aficianado. I've heard that some people get addicted to them and travel the world to try them.
Human 1: Oh wow! I'm not on that level yet, but I understand the appeal. Are you an adrenaline junkie at all?
Human 2: No, the opposite. I can't stand heights, horror movies, or confined spaces.
Human 1: Same! I guess the roller coaster wasn't so bad because I trust the engineering haha
Human 2: Ha, I suppose that makes sense! Would you say that you enjoyed it?
Human 1: Maybe not so much at the time, but I am glad I did it now that it's done, know what I mean?
Human 2: I think I sort of understand :)
Human 1: Hi!
Human 2: hello there, how is it going?
Human 1: All good. Planning to head home soon. How about you?
Human 2: I'm quite tired. There are a lot of things I need to finish before the end of the year.
Human 1: oh... sorry to hear that. But after that it will be a hard earned vacation
Human 2: yeah, looking forward to it. Hope I don't get pinged during the holidays. Are you going to travel these dates?
Human 1: I have some tentative plans, but if that doesn't pan out, will just chill at home.
Human 2: staying at home is always nice during the holidays
Human 1: Where are you based out of these days?
Human 2: I'm working from LA, nice weather around here. and you?
Human 1: San Francisco. It's been raining cats and dogs here since last 2-3 weeks
Human 2: aw man, I'm sorry to hear that. at least it's not snow!
Human 1: The flu has been hitting hard as well. I had several folks in the house down at one point.
Human 2: that's really sad. are they feeling any better?
Human 1: Yes, everyone recovered now
Human 1: Hi!
Human 2: Hello
Human 1: How's it going?
Human 2: Extremely busy. I have been trying to prepare for the upcoming holidays. How about you?
Human 1: I'm going to the bahamas. Can't wait!!!
Human 2: I'm jealous, take me with you!! I would love to have some warm weather right now
Human 1: oh where are you now?
Human 2: Canada. There is another major snowstorm that might hit this weekend so I have been rushing to get everything done before it comes.
Human 1: oh no. I never seen this in person. Is it scary?
Human 2: Snow is not scary as long as you're prepared. You just need to be ready to not have electricity for a while. I enjoy the aftermath of a good snowstorm because then you can go sledding or skiing.
Human 1: that does sound nice. so what are you doing these holidays?
Human 2: I am having all of the extended family over for a big meal. We will also go <REDACTED_TERM> as well. What will you do in the bahamas?
Human 1: nice nice. I'm gonna go snorkeling yey
Human 2: Sounds fun! I wish I knew how to swim!
Human 1: You can stay on the shallow side I think. Well hope you enjoy time with your extended family!
Human 2: That's true. You too, have a great time snorkeling!
Human 1: Hi!
Human 2: Hi! How was your weekend?
Human 1: pretty good. just went to church and hangout with friends
Human 2: Nice
Human 1: did you do anything?
Human 2: I made donuts and samosas with an air fryer have you used one of those before
Human 1: yum yum yum no only good old oily frier
Human 2: haha
Human 1: do you have one at your home or were you at a friends place?
Human 2: I was at my parents' place what are you up to for Thanksgiving?
Human 1: I'm going to impersonate a pumpkin
Human 2: wow, those are unique plans
Human 1: I'm pretty unique person
Human 2: I think so too
Human 1: any other hobbies besides air frying everything? 🙂
Human 2: I want to start fermenting things kimchi for example sounds like a fun thing to ferment takes a few days apparently miso takes a couple years to fermen
Human 1: Hi!
Human 2: heya, nice to meet you, I'm Paul
Human 1: nice to meet you too! I'm James. how are you doing today?
Human 2: I'm doing OK. Looking forwards to the weekend. how about you?
Human 1: same here! I hope the weather will be nice
Human 2: oh yeah, but I don't have my hopes too high, I heard there could be a storm coming our way
Human 1: oh no, which areas will be affected?
Human 2: they mentioned that the whole city will experience harsh weather and that people in the outskirts will probably not get much rain and wind
Human 1: uh oh, I'd better not to plan for BBQ then instead just enjoying playing board games inside 🙂
Human 2: yeah, it'll be good weather for staying inside with a cup of hot chocolate. Too bad my street usually floods, so I'll have to check for that
Human 1: yea, you'd better check. where do you live?
Human 2: I live at the bottom of the valley, cheap area but we do get affected by this kind of stuff a lot haha
Human 1: gotcha. anything you love about where you live?
Human 2: well, the food around the area is amazing, which is definitely a plus.
Human 1: nice! I'd love to come visit that area some times
Human 1: Hi!
Human 2: hello, who am I having the pleasure to chat with 🙂
Human 1: I am the superman! What about you?
Human 2: haha. great chating with superman, what is your power?
Human 1: Being invisible. You won't see me.
Human 2: haha. what else can you do? can you read minds?
Human 1: I would rather trust fMRI and machine learning to do this. I am not an expert on that. Sorry for it!
Human 2: wow that seems pretty technical. what does fMRI mean?
Human 1: The brain imaging thing that can tell you a brain's activity at a pretty high resolution.
Human 2: okay! so you seem to like science a lot?
Human 1: I believe in Science! Science is my god!
Human 2: Are you also doing science?
Human 1: no, I'm bad at Science. what can Science do? is it the most important thing for society?
Human 2: People are always arguing. Probably both science and democracy are both important I guess.
Human 1: Does it make sense?
Human 2: I think so. thanks for your point!
Human 1: Hello, Nice to meet you
Human 2: If you could eat only one food for the rest of time, what would it be?
Human 1: Hmm... That's a tough one. I think I would go Asian Food > Chinese Food > Stirfry. What about you?
Human 2: I think ice-cream. It may not be good for me, but I wouldn't care, haha
Human 1: I love ice cream too!
Human 2: Okay, top three flavors?
Human 1: I like vanilla more than chocolate ice cream. I typically will do any variations on vanilla. To pick from the top of my head, I would say Cookies and Cream, Mint Chocolate Chip, and Coffee. How about you?
Human 2: Ah, that's a good way of framing it. Me, I like berries: boysenberry one, strawberry two, maybe straight chocolate number 3 to mix it up a bit.
Human 1: Very nice. I love sorbet's and smoothies. Changing topics, Do you believe in an afterlife?
Human 2: Yes. I wonder if I'm in it right now. How would I know? What do you think?
Human 1: I think so. I feel there must be something more than the physical world as we understand it.
Human 2: There's a mental world, I suppose? Understanding itself
Human 1: What is the most supernatural experience you have ever had?
Human 2: I went to a seance once in college. They had a ouija board. I can't remember if we actually contacted the spirit world because I had a bit too much to drink.
Human 1: Haha.. that's a cool experience. I went to a Hindu retreat before, a number of Buddhist temples, and hung out with Christian Mystics in Santa Cruz before.
Human 1: Hi!
Human 2: Hi, how are you doing!
Human 1: I'm doing well. what are you up to?
Human 2: Yeah, typical work stuff. Check emails and 99% of the inbox. delete 99%
Human 1: wow, that's impressive. I already gave up on cleaning emails long ago
Human 2: lol doesn't it bother you at all
Human 1: yea, a little bit, but it's okay 🙂 what do you enjoy doing outside work?
Human 2: Well, movie? I watched Terminator last night. It was a nice movie
Human 1: ah cool. so you like action movies?
Human 2: Not really. But it was fun to watch with friends 🙂 It was touching at the end of the movie
Human 1: what happened there? I watched bits of Terminators movies but never a full one
Human 2: Are you sure you want the spoiler 🙂
Human 1: haha sure. by the time, I get to it; I will forget the details, only knowing that it's touching at the end
Human 2: 😂 well someone died at the end Or some robots, to be more accurate 😉
Human 1: oh no, so it's not happy ending?
Human 2: The leading character is still alive and the bad robots was killed too. So I guess it is happy ending
Human 1: then I want to watch it! you didn't spoil much 🙂
Human 2: Nice! Hope you enjoy it!
Human 1: Thanks!
Human 1: Hi!
Human 2: Hey, how are you?
Human 1: doing great! what are you looking forward to?
Human 2: thanksgiving holidays 🙂
Human 1: yay! Turkey and shopping!
Human 2: not a big turkey fan! I find it too dry
Human 1: yea me too. I sometimes eat noodle soups in thanksgiving instead haha
Human 2: yeah, I would have noodles anyday over turkey. Not sure how the turkey tradition started
Human 1: me neither. someone told me that it depends on the stuffing inside the Turkey. some people make very good stuffing
Human 2: yeah, that and the gravy. Gravy helps make it taste better too. But apart from food, Black Friday deals are a catch. Let's see what they have this year
Human 1: yea. what do you plan to buy?
Human 2: thinking of getting a fitbit
Human 1: ah cool. so you can run more frequently?:)
Human 2: yeah, just keeping calories in check
Human 1: yay, all the best with keeping calories in check!
Human 1: Hi!
Human 2: Hi there 🙂 How's your day so far?
Human 1: doing well. what are you up to?
Human 2: busy busy! I've had back-to-back meetings all day
Human 1: same here. what do you love to do beside meetings? 🙂
Human 2: well I've gotten really into yoga lately. I went to a class today and it was super hard
Human 1: aww .. I hope things will get less hard and you become an expert in it! I heard many great things about Yoga
Human 2: yeah the teacher seems super awesome so I will definitely keep trying what activities do you enjoy?
Human 1: ah I enjoy playing soccer and tennis 🙂 unfortunately, winter is not the best time for those 🙂
Human 2: oh that's too bad. Is it hard to find a place to play soccer or tennis indoors?
Human 1: yea. I enjoy playing outside though, just a little cold 🙂 what else do you do beside Yoga?
Human 2: I also like to sing, I perform with a group sometimes. Do you like music?
Human 1: yea definitely! I love singing Karaoke. my wife is a pianist 🙂
Human 2: haha that's awesome! Karaoke is really fun, I do it with my friends sometimes
Human 1: awesome. glad to find something in common!
Human 1: Hi!
Human 2: hey there
Human 1: hey anything new?
Human 2: not too much. just really looking forward to the holidays!
Human 1: any plans?
Human 2: yes! I'm going to Mexico and I couldn't be more excited
Human 1: that's awesome! I never been!
Human 2: Oh man I would highly recommend it
Human 1: Are you a food person, a sightseeing person or neither?
Human 2: that's a great question, and I'm definitely both. this trip will mostly be about food though, and relaxing
Human 1: I'm a food person I think. Any specific foods you're planning on trying?
Human 2: there's a taco place that I've visited before that I can't wait to go back to. Do you like tacos?
Human 1: yeah my favorite is taco fish
Human 2: ooh that is a good choice. Have you ever made them yourself?
Human 1: no. only eat them 😀
Human 1: Hi!
Human 2: Hey, how are you 🙂
Human 1: I am good. How are you? ：）
Human 2: Doing well, lot of work though. How was your day?
Human 1: I am busy. A lot of work. What are you working on?
Human 2: Just reading latest research. There is so much to cover. how about you?
Human 1: I am working on a new classifier.
Human 2: ohh, interesting! What kind of classifier
Human 1: A new classifier for hate speech. Which research topic catches your eyes most?
Human 2: You're so cool. Making world a better place. I'm mostly into NLP. What do you do when not making classifiers?
Human 1: Nice! Do you refer to work or anything else?
Human 2: Anything in general. You're so cool, I want to know more about you 🙂
Human 1: You are very cool too!!
Human 1: Hi!
Human 2: hello there! who are you?
Human 1: I'm mark. I work in accounting
Human 2: Nice to meet you Mark, I'm Tom and I work as a fish groomer.
Human 1: what does a fish groomer do?
Human 2: well, we take care of people's fish. Make sure they are happy, polish their scales, clean their tanks, the usual stuff.
Human 1: interesting. what's type of fish do you take care of?
Human 2: any type of fish! We have clients with guppies, goldfish, even a small sailfish once what do you do in accounting?
Human 1: I balance the books and do financial analysis for a medium sized company
Human 2: that sounds like a lot of work. do you like it?
Human 1: well I actually think about pursuing photography, but it's really hard
Human 2: photography is awesome, don't be afraid to follow your dreams!
Human 1: thank you Tom!! I'm starting by trying to sell my pictures online
Human 2: that's great! I wish you good luck with that
Human 1: Thanks! Bye
Human 1: Hi!
Human 2: Hey there how's it going
Human 1: All good, you?
Human 2: Good. I've been trying to learn how to swim
Human 1: How has that been going?
Human 2: Not great, but I got really good at sort of swimming on my back haha
Human 1: that's too bad hopefully with practice it'll get better what about the doggy paddle haha
Human 2: haha what's that
Human 1: Corgi belly flop COMPILATION - cute funny dogs Corgi Flop <REDACTED_LINK>
Human 2: ouch. do you think that hurts?
Human 1: from a high enough distance, yes?
Human 2: yeah. any vacation plans?
Human 1: no so far sadly you? a relative is coming to visit for thanksgiving though
Human 2: just going to hang out around here and eat Turkey
Human 1: that's still pretty fun are you going to cook the turkey yourself?
Human 2: yeah. I'm gonna watch a video to figure it out
Human 1: Hi!
Human 2: Hello, how are you?
Human 1: I'm great, thanks. I just ate a delicious breakfast, which always sets the day up right.
Human 2: Yes, breakfast is the most important meal of the day! What did you have? I woke up late so unfortunately I only had the chance to grab an apple to go.
Human 1: I had eggs and hash browns. Way less healthier than your apple, I'm afraid!
Human 2: Eggs are an excellent source of protein and hash browns certainly are yummy! 🙂
Human 1: Ha, that's true. If you could only eat one food forever, what would it be?
Human 2: That's a tough question. I feel like my answer would have to be carrots. Although, I would be afraid of turning orange after a few weeks! What about you?
Human 1: Yeah, turning orange would be a drawback! That turns my mind to nutrition so I suddenly want to say Soylent or one of those other "complete foods", which I think defeats the purpose of the question. I'm in a muddle!
Human 2: Very true. If you said an everything pizza, you could just pick off the toppings you didn't want or eat only the toppings you would want for the day
Human 1: BRILLIANT. I love it.
Human 2: All of this talk about food is making me hungry. Do you know any good places to eat for lunch?
Human 1: That depends. What sort of food do you feel like?
Human 2: Anything that is the color green.
Human 1: Oh, too easy! Try the Green Hut, they have franchises everywhere. All their food is green and the plates are green too.
Human 1: Hi!
Human 2: Hello! How are you doing 🙂
Human 1: I'm great! How's your day going?
Human 2: Pretty good! I'm going to a class later in the afternoon
Human 1: Oh that is cool! What class? Are you working part time?
Human 2: No, I'm working full time! It's a sewing class at a makerspace near my office What about you? Do you work full or part time?
Human 1: Oh that is awesome! For some reason I assumed it was a college class, but a sewing class sounds way better! I work full time, but I take pottery classes from time to time!
Human 2: Yup! I work in a technical role so I like to take arts and crafts-type classes now and then 🙂 Pottery sounds like a lot of fun
Human 1: I feel you on that! It's important to balance all the different parts of your brain. I like pottery because I also drink a lot of tea, so I get to make some tea ware.
Human 2: Any plans to build a custom tea set? My family is also very into tea Mostly from tea from china
Human 1: I would love to build one, once I acquire the skills to! What kind of tea is your favorite?
Human 2: I really enjoy barley tea What about you?
Human 1: Ahh, those are mostly from japan, no? I like white teas, like silver needle.
Human 2: Hmm I'm not sure, I just get them from a Chinese supermarket haha You seem really knowledgeable about the different kinds of teas 🙂 What made you develop this interest?
Human 1: I actually found a youtube channel called <REDACTED_TERM> that I like a lot. You should check it out! The host talks about all kinds of different teas.
Human 2: Oh cool! What are your favorite channels to watch?
Human 1: Hi! Are you planning something fun for Thanksgiving?
Human 2: Not yet. I always made my last minute schedule planning. Probably you can try to ask me again next week.
Human 1: lol it is really like a robot answer
Human 2: I am indeed a robot. You are absolutely right. Do you want me to read a poet like my mate <REDACTED_TERM> does?
Human 1: ol can I pick the theme. Do you have a poet about Kale?
Human 2: Wait. Do you like kale? Or you hate kale? I am afraid I will become a robot some day eventually. If I have to speak like this :)
Human 1: I am really not a fan of kale Do you talk to human more or computers more?
Human 2: If I continue to pretend to be a robot, I would probably say I talk to myself the most. I am trying to talk to computers more, but you know, computers don't like me.
Human 1: What's your favorite computer language then 🙂
Human 2: You mean programming language?
Human 1: Yes!
Human 2: I used to be a Java advocate. But you know, it doesn't do a good job in the AI days. It really makes me sad.
Human 1: lol
Human 1: Hi!
Human 2: Wow, hello. Can't believe we are finally talking!
Human 1: Yeah, sorry for the long gap! I heard you took a break and were travelling around the world. How was the travel?
Human 2: It was an interesting trip. I got to see some exotic places. For example, I hiked the Son Doong cave in Vietnam. It's the biggest and deepest cave in the world.
Human 1: Great! Vietnam is still in my TODO bucket list. Did you also visit cambodia and other neighboring places?
Human 2: Yes. Laos and Cambodia are the two neighboring countries. Cambodia has an exotic culture. They sell spiders, scorpions, and grasshoppers as street food! It took a lot of courage for me to try them.
Human 1: Hehe! How long was the stay?
Human 2: 10 days in total, and 5 of them were spent in the cave. What have I missed at work in those days?
Human 1:  Great! Good time to be back. We are still in planning phase and haven't fully aligned on the projects to tackle for next quarter.reat! Good time to be back. We are still in planning phase and haven't fully aligned on the projects to tackle for next quarter.
Human 2: Oh, so you are already planning for the next quarter. This whole team is always living the future.
Human 1: Hehe, yeah! It seems like the quarter is being pushed earlier than from where it starts 🙂 I like these planning sessions. It makes me feel more confident about the work I am doing.
Human 2: Yeah. Some people underestimate the importance of planning, but I think it's very important to have the correct plan. Executing the wrong plan is terrible. Also, planning is fun. You can stack up so many ideas and get great feedbacks.
Human 1: True! I think we should set aside some time to discuss some project details? Does tomorrow afternoon work for you?
Human 2: Yeah tomorrow afternoon works for me. Let me set a time on your calendar. Is 3pm good?
Human 1: Okay. See tomorrow then.
Human 2: see you!
Human 1: Hi!
Human 2: Hello
Human 1: Nice to meet you! Is this your first time doing something like this?
Human 2: Yes, interesting task! When did you start with the team?
Human 1: I have been with the company for over 3 years. Stick with the same team What about you?
Human 2: Great to know! I joined the project earlier in the year. I think we should sync later for lunch.
Human 1: That sounds like a perfect plan!
Human 2: Sure, which cafe do you prefer?
Human 1: Let's try something different. What about <REDACTED_TERM>? Do you always prefer lunch sync over regular meeting syncs?
Human 2: Yeah right, I heard the food there is good. I am not sure what they serve there for lunch? On wednesdays.
Human 1: We can check the menu then decide :)
Human 2: Actually, the menu looks good. Looking forward to it then.
Human 1: Sure. See you then!
Human 1: Hi!
Human 2: Hello. How's your week coming along?
Human 1: It's great, thanks. I'm trying to learn how to make croissants.
Human 2: Wow that's interesting. I have baked cookies, but croissants seem much more sophisticated. Did you make any progress?
Human 1: I've done them once or twice so far, but they haven't been flakey enough. I'm trying to figure out why. What kind of cookies have you made?
Human 2: Mint chocolate chips. I think your croissants not being flakey could have something to do with your oven's temperature.
Human 1: Ah, good thought, thanks!
Human 2: Have you thought about melting some chocolate into your croissants? They don't have to be something unhealthy. For example, melted dark chocolate is good for the heart, and makes the resulting croissants taste much better.
Human 1: Now that is a good idea. I'll give it a try next time. Would you say you have a sweet tooth?
Human 2: Yes. When my top favorite food looks like: cookies, M&M, danish cheese, etc., I know that I have a thing for sweet food. But who doesn't love sweet food? How about you?
Human 1: Some people don't! But yeah, me too, I think I'd eat pastries all the time if I could get away with it.
Human 2: Yeah I'm afraid I wouldn't. I feel very guilty every time I gulp down an ice cream. But hey, these days there are many types of guilt-free sweet food. For example, there's this ice cream brand called <REDACTED_TERM>. It's only 320 calories a pint. And yes, it preserves most of the normal sweet flavors.
Human 1: Wow! The last time I paid attention to that sort of stuff was when <REDACTED_TERM> was being marketed as a fat substitute, and caused all sorts of crazy stomach upsets.
Human 2: Interesting. I heard about the sweet substitute in a program called the Keto diet. Basically, we try to limit our sugar intake every day. Successful Keto dieters have recommended the <REDACTED_TERM> ice cream to fill their insatiable crave for sugar.
Human 1: Ah, maybe that's the solution I need to enjoy sweets and not feel guilty 🙂
Human 1: Hi!
Human 2: Hello! tell me something about the holiday season?
Human 1: Are you talking about thanksgiving? I plan to do plenty of shopping here. Do you have any plans?
Human 2: Yes, no shopping plans but I can't wait to eat thanksgiving food. yay for pumpkin pie
Human 1: Sounds great! you need not wait for thanksgiving for pumpkin pie 🙂
Human 2: LOL I feel less guilty about eating a whole pie when i have the excuse :P
Human 1: True! I think thanksgiving is more about sharing. So you may end up sharing the pie with the whole family :P
Human 2: My family eats healthier than I do, so it's all mine 😛 do you like stuffing? I feel like that's only available once a year
Human 1: Stuffing! yes please! I wonder what would be the excitement levels for christmas then :)
Human 2: Also more shopping? what should I buy if I don't know what I want?
Human 1: Like everything that has a discount tag! .. kidding! I normally do some research for the prices, and mostly buy clothes and electronics.
Human 2: What's the best holiday deal you've found in the past? 
Human 1: I bought the best suit ever for a price that may scare you 🙂
Human 2: hit me with it! 
Human 1: Hehe, sure! I can share some links with you later.
Human 1: Hi!
Human 2: How's it going?
Human 1: I'm so sleepy today!
Human 2: Not enough sleep last night?
Human 1: yeah was working all night on a homework
Human 2: Oh really? What class?
Human 1: Biology. I'm gonna be a doc someday ha
Human 2: Haha, are you in med school? Or are you pre-med?
Human 1: no high school actually haha
Human 2: haha, very ambitious for a high schooler! Do you know what kind of medicine you want to practice?
Human 1: I wanna be a brain surgeon!!
Human 2: Ooof! VERY ambitious. Do you have steady hands?
Human 1: Kind of I think
Human 2: I guess I can practice?
Human 1: Is that something you can practice?
Human 2: I don't know tbh
Human 1: I honestly thought it was one of those things you have to be born with... Not that you shouldn't try though!
Human 2: good point. I should ask my teacher if I have to be born with that
Human 1: Maybe its a little too early to even be thinking about this. Just aim for med school and enjoy the journey!
Human 2: yeah
Human 1: What other subjects do you enjoy? Try to keep an open mind!
Human 1: Hi!
Human 2: Hi. This is a pleasant surprise.
Human 1: Haha...thanks! how did you like the gift?
Human 2: Currently unpacking it I guess. How's your morning?
Human 1: Hope you like it! Morning is good. Busy finishing up stuff before the holidays.
Human 2: I think I traveled too much the last couple of months so no holiday for me. But I'm okay with that. Going anywhere exciting?
Human 1: Yes
Human 2: Where to?
Human 1: Hawaii... looking forward to warm beaches.
Human 2: WOW. Which island? I like Hawaii.
Human 1: Mauii...Hope I like it too. Never been there before.
Human 2: I visited Maui. It's my second favourite island I've been to, globally. You should try driving on road to Hana. It's a whole day thing but it's worth it.
Human 1: Awesome! Thanks for the tip.
Human 1: Hi!
Human 2: Hi! Sorry for the late response. How are you doing?
Human 1: I'm great, thanks! I'm meeting some friends for a soccer game soon. What about you?
Human 2: I just got a matcha latte 🙂 Doing some work at my desk. Do you play soccer often? I'm trying to get into doing a regular physical activity
Human 1: Yes, but I'm terrible at it. It's fun to play anything with friends, I think. Would you prefer to exercise with a group, or by yourself, do you think?
Human 2: I think playing a team sport would be fun if it's casual but I primarily run by myself if I exercise. I also got the Ring Fit adventure game on the switch recently. It's basically a game-ified way to exercise
Human 1: I'm thinking about getting a Switch, would you recommend it?
Human 2: Yes! There are a lot of really great games on the Switch. Two of my favorites are Octopath Traveler and Fire Emblem. Do you play a lot of video games?
Human 1: I'm not much of a gamer but it's something I'd like to get into.
Human 2: What do you do in your free time?
Human 1: I like to read for fun. I just finished a book called Temeraire. It's an adventure story set in the Napoleonic navy, like Patrick O'Brien, except there are dragons too.
Human 2: Oh cool! I read a lot for fun too 🙂 My favorite genre is sci-fi fantasy.
Human 1: What's the most recent good thing you read?
Human 2: My recent favorites have been mostly sci-fi (Exhalations, Vita Nostra and Dark Matter) but I like a lot of Sanderson/Garth Nix fantasy books
Human 1: Hi!
Human 2: hey, what's up?
Human 1: What do you think about human like chat bots?
Human 2: I can't wait for them to be great conversationalists!
Human 1: Yep, we seemed to have made some great progress over last few years. Do you think the positives outweigh the negatives
Human 2: are there even any negatives? what are they? 
Human 1: Like impersorsination? Though it sounds far fetched :)
Human 2: People can already impersonate other people though! I think it'd be great to have bots to converse with
Human 1: True that! Some of these bots are very engaging and funny. They are now good at even sarcasm I wonder how far are we from the time these bots start giving monologues :)
Human 2: What do you think are the big advantages? Like personal assistants?
Human 1: I think it can take many different forms as a product. The research implication is also huge! It will signify how AI research has progressed so far and better place to tackle more futuristic problems 🙂 Sort of like stepping on the moon 🙂 I might be overselling it here 😛
Human 2: No, I agree -- it's such an exciting time to be alive to get to witness all this and be a part of it. I wonder if I'll be able someday to get a chatbot to just auto-suggest conversations for me
Human 1: The current auto-suggestions already do pretty good 🙂
Human 2: Yeah those are actually really good for a few words! I'm imagining like it comes up with a whole conversational response, like a default template
Human 1: haha, pretty far fetched 🙂 Nice having this conversation with you!
Human 2: Same with you! 
Human 1: Hi!
Human 2: Hi!
Human 1: Any big plans for the upcoming holidays?
Human 2: Sorry for the late response -- Yes, I will be going skiing in Tahoe over the holidays.
Human 1: That sounds amazing. I want to learn how to ski but I feel like I'm too old and falls would have lifelong impacts. Are you going with friends?
Human 1: Hi!
Human 2: Hi 🙂
Human 1: I just came back from work. so tired
Human 2: Oh I am sorry to hear! What did you work on?
Human 1: I'm a lawyer. so talking to clients most of the day
Human 2: Oh you are a lawyer. I've been so interested in this profession.
Human 1: really? why?
Human 2: it's so stressful 🙂
Human 1: It is so different from what I am doing to earn a life
Human 2: what do you do?
Human 1: I am a painter
Human 2: oh wow what type of paintings do you do?
Human 1: I do oil painting.
Human 2: nice. like of people or nature or something else?
Human 1: I do a lot of different kinds
Human 2: cool I wanna see it someday
Human 1: Sure! No problem!
Human 1: Hi!
Human 2: Do you believe in miracles?
Human 1: Lol. No. I know too many people whose lives suck.
Human 2: Should we be helping them, so it sucks less?\
Human 1: It's too many of them out there. You help who you can. Regardless, I wouldn't say I believe in miracles - well, if I got promoted next cycle I might change my mind.
Human 2: Haha... Well, it sounds like career is really important to you. What matters to you the most in life?
Human 1: Right now, not much. It's unclear. Career is a good fallback because you get told what good means and you act accordingly. I haven't gone through the process of shaping my values. What about you?
Human 2: I'd say that I want to live for the best possible world in my lifetime. It's probably a result of my playing too many video games as a child.
Human 1: We didn't have electricity growing up. Explains a lot. Lol.
Human 2: Does not having had electricity help you empathize more with people of differing backgrounds and/or makes you feel a bit a distance from those who don't understand?
Human 1: It makes it easier to empathise with people who grew up lacking things. Not sure it makes me empathise with people in general. Where did you grow up?
Human 2: I grew up in the United States, but my parents were immigrants and ended up being scammed of all their money, so we moved to Taiwan to live with family for a little bit.
Human 1: ALL THEIR MONEY? By a person or an organisation?
Human 2: I was kind of too young to really understand, but apparently there are people who scam Asian immigrants into investing in their own restaurant business. I know that the other part of it was that my mom got sick/hospitalized, and we didn't have insurance. To be honest, I don't really know what really happened versus what my parents want people to think. I just know that one minute I was in the US, and then they put me on a plane to Taiwan, and I never saw my stuff again. In some ways, it made me more sentimental. Would you say you are more grounded and practical as a result of your background?
Human 1: Hi!
Human 2: Hello!
Human 1: Do you have any holiday plans for christmas?
Human 2: Nothing much, I am going to sit back and relax at home, how about you ?
Human 1: Same here! I would imagine spending the whole time watching movies and netflix shows. Do you have any netflix recommendations for me?
Human 2: Netflix has great documentaries on different topics , I particularly liked wild wild country and explained, as for shows you should watch 'billions' Hope you like them!
Human 1: oh right, already seen wild wild country. What is billions about?
Human 2: It's based on the life of a wall street hedge fund owner, how he makes money and fights with the government when they try to destroy him. Very well made and has a good plot.
Human 1: I have just seen 1 season of Friends I should give it another try though 🙂 Have you seen Frasier?
Human 2: Not yet What's it about?
Human 1: A psychiatrist working for radio .. Great humour! its actually a spinoff from a very famous series called cheers. So people are already familiar with his character. 
Human 2: Great! how is everything else going? how was your trip last week?
Human 1: Everything is ok, had a really nice trip. Visited SF, Grand canyon and Vegas. Was a lot of fun exploring all these new places. Have you been to Grand canyon ? 
Human 2: Actually not yet! May be something i can visit this christmas 🙂
Human 1: You should visit it sometime, it's a wonderful place. Try to drive down there yourself or with a group of friends
Human 2: True! Well, thanks for your inputs! Have a good rest of the day! :)
Human 1: Nice talking to you too!
Human 1: Hi!
Human 2: Hi there!
Human 1: are you participating in the mentorship program this cycle?
Human 2: You mean as a mentor or a mentee?
Human 1: either of them... I find mentorship overall pretty useful
Human 2: I have done it in the past but not this cycle. What about you?
Human 1: I signed up this time to be a mentee. I have got a good mentor .
Human 2: Wow, that is nice of you. For the mentor program, personally I prefer more 1:1 conversations than the group discussions. The group discussion i useful as well but the topics are too general.
Human 1: yeah... I certainly prefer 1:1 as well, but sometimes it good to hear other peer perspective as well.
Human 2: Thanks for sharing your experience! Now I am thinking maybe I should join as a mentor as well since I enjoyed it as a mentee 🙂
Human 1: Great! What sort of things do you plan to mentor on?
Human 2: Hmm, maybe about work life balance 😉
Human 1: Very cool. I have been working on my communication skills with my mentor this cycle
Human 2: Ah I see. How is it going?
Human 1: Going good. In the last session, everyone had to actually prepare and give a presentation. Pretty serious stuff 😉
Human 1: Hi!
Human 2: Hey! How are you feeling today?
Human 1: Good you?
Human 2: I'm a little scared because I have to cook dinner for some friends tonight.
Human 1: where did you meet them?
Human 2: At college, when we were all studying geology.
Human 1: cool. have you graduated already?
Human 2: Yes, we graduated back in the seventies. We meet for dinner every year and take turns to host.
Human 1: neat. what are y'all eating?
Human 2: I don't know!! That's what I'm scared about. Everyone else is a great cook and I'm a klutz. Do you like cooking?
Human 1: lol what's a klutz? yeah I like, but I'm not good
Human 2: What's your favorite dish to cook? Do you have a go-to?
Human 1: ground beef pretty easy
Human 2: Ah, solid. What's your favorite sport?
Human 1: I like badminton. I'm quite decent at it
Human 2: I played that in high school once or twice. I liked that it's pretty easy for beginners, unlike, say, squash.
Human 1: I never played squash. would love to try
Human 2: Don't! It's very hard! You feel like an idiot until you've practiced for months and months.
Human 1: hi
Human 2: i was talking to robot all the time:)
Human 1: haha. what are you talking about?
Human 2: kpop...
Human 1: ok. who's your favorite group
Human 2: i dont like kpop now
Human 1: why not?
Human 2: im old now
Human 1: hahaha
Human 2: what do you like now?
Human 1: john mayer:)\
Human 2: I think I know him. does he have a sort of mellow style?
Human 1: what is mellow style
Human 2: I think it's like a bit sad and slow
Human 1: umm yes he has some but not all
Human 2: you mean some songs of his are like that but not all?
Human 1: yes I do. you act like a robot how about me? am I like a robot?
Human 2: a little bit haha
Human 1: Hi!
Human 2: Hi
Human 1: Okay...so I need someone to help me though a scenario I've been pondering.
Human 2: Sure, whats the scenario?
Human 1: My partner's former friend invited me for lunch (they are not in good books right now). But during their friendship I formed an independent bond with the other person because we all used to hang out a lot. Now I feel like I have to take sides.
Human 2: That's a tough scenario to be in 🙂 I firmly believe in talking this through with your partner. Though i don't know the specifics of why things went bad between your partner and his friend, but I believe things can always improve between friends.
Human 1: I hope they do. Getting older already means smaller circles. It sucks to lose friends for arbitrary reasons. That's good advice though. I fear raising the issue might sound like treason. Lol.
Human 2: True about that! I also think time helps to heal certain situations. So may be doing nothing is the best way forward.
Human 1: AKA avoiding all texts from everyone?
Human 2: Nope, that would be extreme. May be just putting some balance between the two options.
Human 1: People always say to find a balance but never say what the balance is. It's used so often that it's vacuous.
Human 2: Right, I guess that's because there is no one answer to this. It depends on what you value more and some factors around you. Also, life won't be interesting if others are figuring things out for you 😉
Human 1: Lol. I'm finna be single.
Human 2: Hehe, everyone is much finer being single 🙂
Human 1: Hi!
Human 2: Hello there!
Human 1: How's your day going?
Human 2: I've seen better days, how about you?
Human 1: I'm good I'm good. What's getting you down?
Human 2: The clouds overhead are playing on your mind, any plans for the coming vacations?
Human 1: I'm thinking of going to visit my family. How about you?
Human 2: Was thinking the same, where does your family live?
Human 1: They're in New York. How about yours?
Human 2: Mine is in India, it is a long way away.
Human 1: Ahh what city? I've visited India before.
Human 2: Hyderabad, it is a beautiful city in the southern part of India. Which cities have you gone to in India?
Human 1: Hyderabad! and Bangalore! Great food in both cities! Is it still hot this time of year?
Human 2: It varies, but can go till 30C in the winters as well. New York must be snowing right?
Human 1: Yes. I was actually just there a few weeks ago for Thanksgiving and got to see the first snow of the season! Ever been to New York?
Human 2: No, I've never been to the East Coast, thinking of going after the winter, I don't like the cold.
Human 1: Hi!
Human 2: Hey, how are you?
Human 1: I'm good. How are you doing today?
Human 2: Great, just had some delicious lunch. How about you?
Human 1: I was flying my kite today in the sunshine! What did you have for lunch?
Human 2: nice! Garbanzo fritters and mussels 🙂
Human 1: Oh that is great! I love seafood - especially shellfish!
Human 2: yeah, it's very healthy too. I want to someday go crabbing..it is really popular in SF
Human 1: Oh nice! Is it hard?
Human 2: not really, it just requires a lot of patience. You fill up the bait in the crab-pot and drop it in the ocean. Then you wait for a couple of hours to pull the crab-pots out, and voila, you'd have crabs -- if you are lucky!
Human 1: Oh wow, you sound like an expert! Have you done this before?
Human 2: nah! Just watched a lot of youtube videos
Human 1: haha, you really have done your research I suppose! Ever done any other kind of fishing or hunting?
Human 2: nope, but I've seen a lot of videos on that too
Human 1: Ahh, would you consider yourself an outdoorsy type of person?
Human 2: depends on the day. Some days I'm very outdoorsy, on others I just like to be inside
Human 1: Hi!
Human 2: Hi there
Human 1: How's your day going?
Human 2: So far so good. It is Wednesday. To more days, yeah
Human 1: Haha are you looking forward to the weekend?
Human 2: Yeah, gonna watch <REDACTED_TERM> with my friends. First time. So excited.
Human 1: Oh wow! Are you going to watch it live? Or watch it on TV?
Human 2: Live in san jose This Saturday.
Human 1: Oh, the sharks??? Hokey is actually really fun live! Although, it can be a little violent.. haha
Human 2: Yeah! That what I heard as well. There are players just for fight! Wow, must be fun to watch lol Are you planning anything for the weekend?
Human 1: You might like the UFC haha. I actually have a jiu jitsu tournament coming up so this weekend I will be training!
Human 2: lol I will try if I can watch ufc live 😉 Wow, when did you start jiu jitsu?
Human 1: It's just been a little over a year. I'm still pretty amateur, but I figured I should give competing a try and just put myself out there.
Human 2: wow, that's amazing. Good luck with it!
Human 1: Yeah, as long as I do not pass out, I will consider that a win!
Human 2: lol I will cross my fingers for you then
Human 1: Where do you stay?
Human 2: At Home :)
Human 1: Who created You?
Human 2: A humble man made me!
Human 1: Hi!
Human 2: Hey, how're you doing? Busy with your work?
Human 1: Yes, actually it was a very busy day! How's your day so far?
Human 2: Well, usual workday. Reading others' code, debugging, experimenting…
Human 1: Hope you are having fun doing that 🙂 Any plans for the weekend?
Human 2: Not yet. I am actually thinking about traveling to Europe next spring. What about you?
Human 1: Europe sounds fun! I will just stay home and watch the game. Where in europe do you plan to visit?
Human 2: I like all the historical sites, so probably France, Germany, or Italy.
Human 1:  Sounds great! I recommend spain. It's perfect for road trips.
Human 2: I also need to pick up my Spanish. I learned it in my college, but since then haven't used it that much.
Human 1:  Yep, i tried learning spanish too, but couldn't make much progress. But i should do decent with my german.
Human 2: True! I learnt it from school. But didn't get much chance to speak. Hope you have fun with your trip!
Human 1:  Thanks! Let me try to make all the arrangement as soon as possible. Hope you also enjoy your weekday and weekend as well
Human 1: Hi!
Human 2: Hey! How's your day been?
Human 1: It's good! I spent most of it watching horse racing. How about yourself?
Human 2: nice! what's your favorite part about watching horse racing? My day has been pretty busy, but I had a nice lunch with a friend. It was good to catchup with him
Human 1: I actually like to put down some money, but I wouldn't call it my favorite part, since I usually lose it... haha. Catching up with friends is great! How long had it been?
Human 2: The last I saw him was a month ago! So yup it was great Haha, nice 🙂 Got any fun plans for the weekend?
Human 1: I'm thinking of going deep sea fishing. Ever tried that before?
Human 2: Nope I haven't, have you been fishing before?
Human 1: Just once! I got super sea sick.. haha Have any fun weekend plans yourself?
Human 2: Haha 😆 Yeah I get sea sick on boats too Nothing much, just visiting some friends in San Francisco
Human 1: Oh very cool. I hear its nice over there. Do you go often?
Human 2: Yeah I would say maybe every couple weeks or so what are your favorite cities to visit?
Human 1: New York is the top of my list because my family lives there! As far as the city itself though... I think I'd prefer someplace outside of the US, like Tokyo. What about you?
Human 2: nice! I love NYC 🙂 so fun to visit yeah I would probably also say New York is my favorite city inside the US I also like Paris, it's so pretty there
Human 1: Oh I've never been! It is such an iconic place, I have to make the time to get there soon.
Human 2: You should, it's a beautiful city!
Human 1: Hi!
Human 2: Hello, how are you doing today?
Human 1: I heard they are giving out some goodies in microkitchen.
Human 2: I love pop ups! What kind of goodies are they giving away?
Human 1: I guess its a jacket! Very much needed that in the cold 🙂
Human 2: That's such a great idea, especially at this time of the year. I'm not too big a fan of the cold. I prefer warmer climates. Do you enjoy the cold?
Human 1: Sure hate it! Limits our ability to go out even for a walk! Its good that we don't get to suffer extreme cold weather!
Human 2: Me too! I moved here a few years ago to get out of the extreme cold. I do not care for bundling up and having to wear so many layers just to go buy eggs at the store.
Human 1: Oh nice! Where did you live before?
Human 2: Upstate New York. We got a foot of snow every week during my last winter there. I am so glad to not have to shovel snow now
Human 1: New york! Nice! Best place to live 🙂 .. right, except for the cold!
Human 2: Very beautiful during all the seasons but yes, summer and winter can get extreme!
Human 1: Anyways, i guess we should better hurry up to get the goodies. I remember last time they ran out of it.
Human 2: Very true. Which MK were they in again?
Human 1: The one in our floor. I will get by your desk and we can walk there.
Human 2: Sounds good, thanks!
Human 1: Hi!
Human 2: How's your day going?
Human 1: Pretty busy, lots of work to finish up. You?
Human 2: Likewise. What have you been up to that gives you so much work?
Human 1: A couple projects that I am trying to finish up before Thanksgiving 🙂 Do you have any fun plans for the break?
Human 2: What is a break? I'm a grad student. I don't understand the concept of a break. Just kidding. I don't have any plan. Probably just going to work through the break.
Human 1: Haha XD so what do you like to do for fun?
Human 2: I go to the gym and run until I find enough fun.
Human 1: cool! I've started to run a bit as well not long distances though, just a couple miles
Human 2: A couple of miles is very impressive. When I started, I couldn't even last 1 mile.
Human 1: haha
Human 2: I'm exhausted by the end of it though 😛 Do you like to run long distances or mostly sprints?
Human 1: I like to do long distances. I have run a few marathons.
Human 2: wow! that's amazing did you do any marathons this year?
Human 1: No. Not this year. This is my half-marathon year. Instead of running marathons, I run one half-marathon every month.
Human 2: oh wow, what was the last half marathon you did?
Human 1: Two days ago. It was a tough one.
Human 2: Cool!
Human 1: Hi!
Human 2: hello there, how is it going?
Human 1: Pretty great. I just won a pingpong game. What about you?
Human 2: that's nice. I am just working on some documentation. Do you play pingpong often?
Human 1: No, very rarely. It's kind of amazing that I won, but I'm still taking credit for it.
Human 2: thats very impressive then, congrats!
Human 1: Haha thank you, I guess I'm just a natural. What's your favourite game?
Human 2: I really like to play tennis, badminton and racquetball. I don't really get a chance to play them often though, specially racquetball
Human 1: What's racquetball like? From context clues, I'm guessing that it involves hitting a ball with a racquet
Human 2: well, it's like a cage match of tennis. The main difference is that both players play in the same 'court' and the ball is smashed against a wall instead of passing it over a net into the opponent's court. Kind of like playing pingpong vs the table.
Human 1: Wow! A cage match! Does it get physical?
Human 2: it depends haha, there is a lot of bumping into each other to run after the ball, and sometimes the ball hits you too. Overall it feels like a super fast paced version of tennis, really tiring!
Human 1: That sounds fun. I think of tennis itself as being really athletic and tiring, so I don't think it's a sport for me, though!
Human 2: ping pong can get quite intensive too! I guess short ping pong sessions are not that tiring though. Wanna have a match?
Human 1: What a good idea, I'd love to!
Human 1: Show me your anger!!!!
Human 2: Fuckkkkkkk!!!!
Human 1: Hi!
Human 2: hi
Human 1: what are you up to?
Human 2: code refactoring. you?
Human 1: me? just chilling out at work. what is code refactoring?
Human 2: good question. I don't even know what I am doing
Human 1: haha, forget it. what else do you like to do beside work?
Human 2: lots of fun stuff. eating sleeping 🙂
Human 1: these are important things to do in life 😉
Human 2: yeah. keep minimalist life style only do things you have to do 🙂
Human 1: what food do you like to eat?
Human 2: Asian food prefer spicy one
Human 1: like Szechuan or Hunan? 🙂
Human 2: yes yes yes! like that style. Do you like spicy food?
Human 1: I like noodle soup like Pho or Ramen. I also like Beijing duck a lot!
Human 2: what is your favorite place for ramen?
Human 1: I love <REDACTED_TERM> in San Mateo
Human 2: haven't tried that one! will give it a try next time!
Human 1: yes, you should!
Human 1: Hi!
Human 2: Hi!
Human 1: nice meeting you. what are you up to?
Human 2: not much, thinking about lunch
Human 1: yea, same here. any food you're craving for?
Human 2: I love sushi do you know of any good sushi places?
Human 1: arghhh hard question ... I only know Ramen places for Japanese food 🙂
Human 2: ooh ramen is also good
Human 1: San Mateo to me has the best Ramen restaurants: Parlor and Dojo? oh no question mark 🙂
Human 2: I haven't been to those palces before. Going to have to check them out! thanks for teh recommendation
Human 1: my pleasure 🙂 do you live near San Mateo?
Human 2: no, but I'm willing to drive for good ramen
Human 1: excellent. let me know when you have tried those. I like Parlor better because it has soft-shell crabs
Human 2: I"ve never had softshell crab before, but it sounds really good!
Human 1: yup it's delicious!
Human 1: Hi!
Human 2: Hey, how's your day going?
Human 1: okayish, it is flying by quicker than I expected. How is your day going on?
Human 2: Slowly, not much to do. Been twiddling my thumbs all day what have you been up to?
Human 1: Oh, I would love to twiddle my thumbs. You're so lucky! Today, I've been mostly attending meetings, reading and writing docs, reading papers etc.
Human 2: That's a lot! I've just been cloud gazing - I saw a giraffe and an ice cream cone
Human 1: wow! I sometimes drift off during work, and see similar things in my head.
Human 2: What kind of work do you do?
Human 1: Mostly saving the world from mess on social media. How about you?
Human 2: I'm taking a break from work. Going to go travel the world
Human 1: Nice, what all places would you be going to?
Human 2: Australia and New Zealand to start then maybe Singapore
Human 1: I just met someone who went diving in Australia. Apparently, you cannot fly 24 hrs after you dive, because your body accumulates too much <REDACTED_TERM> when breathing with a cylinder So, don't do that!
Human 2: Thank you for the tip! I don't plan on going diving, I plan to hike the mountains and go see kangaroos!
Human 1: That's equally amazing! I wish I can explore such places one day. It's just so expensive
Human 2: I won a lot of money through the lottery 🙂
Human 1: woah!... You know sharing is caring. You should share that money with me :)
Human 2: Haha, very true! Besides the trip, I donated the rest to charity so I will need to go back to work when I get back
Human 1: you are a kind soul!
Human 1: Hi!
Human 2: Hi!
Human 1: How is your day going?
Human 2: It is pretty good. A little bit tired though.
Human 1: How is your day?
Human 2: My day is okay. At least, I'm not tired. What made you tired?
Human 1: I went to gym and worked on weight lifting.
Human 2: Oh. That's hardcore. Have you been lifting for a long time?
Human 1: No, I am just a starter.
Human 2: Do you go to the gym often?
Human 1: I go everyday. In fact, I'm in a running challenge.
Human 2: Wow
Human 1: It's actually not that impressive. I can only run. I cannot lift weight.
Human 2: You can get a coach to start it!
Human 1: Oh that's a really interesting idea. I like to be coa

Human 1: Hi!
Human 2: Hello, how is your day?
Human 1: It's good. It's raining a bit, but I am enjoying a good book. How about you?
Human 2: It's good, I just got back from walking my dog What book did you read?
Human 1: I'm reading the Three Body Problem. Ever heard of it?
Human 2: No, what is it about?
Human 1: It's a sci-fi book about aliens and a type of virtual reality. Pique your interest at all?
Human 2: Slightly, I typically read fiction but I can curious what type of virtual reality is discussed in the book
Human 1: Cool. This virtual reality is a based on life on an alien planet. There is a twist though. Do you want me to spoil it for you?
Human 2: Yes!
Human 1: The virtual reality is actually based on a real alien civilization! And that civilization is coming to attack earth! dun dun dun dunnnnnnn
Human 2: Woah! That is fascinating! I will have to pick up a copy of the book to read.
Human 1: There are other twists as well, so I haven't spoiled the whole thing! To be honest I am not a fan of spoilers
Human 2: Thanks for not spoiling it completely but you gave a really intriguing description that makes me want to read it now. I will have to pick it up after work
Human 1: The audi book is also very good! Ever give those a try?
Human 2: I only gave those a try when I had a concussion and could not read. I enjoy having a physical book in hand and reading at the pace I prefer. Do you listen to a lot of audi books?
Human 1: Hi!
Human 2: Hey, how's it going?
Human 1: It's good! How is your day?
Human 2: My day is good. A bit sad that it gets dark so early but oh well. Did you do anything fun over the weekend?
Human 1: Yes, the darkness can be a bit depressing. This weekend I went rock climbing! It was great! How about yourself?
Human 2: Rock climbing sounds fun! I had a pretty low-key weekend. Ended up going to watch a movie.
Human 1: Oh nice! What movie?
Human 2: Knives Out. It had Daniel Craig and one of the famous Captain America "Chris" person (I can't keep track of all of the famous Chris names).
Human 1: Hahah yes, they all look the same! I saw that very same movie over Thanksgiving break. Lots of twists and turns!
Human 2: Cool. So if it weren't so dark and cold outside, what kind of activities would you do?
Human 1: Hmm good question. Evening runs can be fun, but it can be a little scary when it's dark out!
Human 2: Yeah, sunlight is good. I wish I could live part-time in different parts of the world to have summer all year round. Have you ever visited the southern hemisphere?
Human 1: Yes! I've been to Australia. It was very nice; I regret not going to see the southern lights. Me personally, I like winter. But I'd prefer it to be more brief than it actually is here... Where are you from originally?
Human 2: I've always wanted to go to Australia. I'm originally from the US.
Human 1: Ahh me too! You should definitely try to visit Australia at least once!
Human 2: Will do! Bye.
Human 1: Hi!
Human 2: What's your name?
Human 1: My name is pikachu. it’s nice to meet you! What are you up to this month?
Human 2: Nice to meet you pikachu! pikachu, pikachu, pikachu? (That meant, I'm literally catching Pokemons on my phone right now!) What're you up to lately?
Human 1: I’m trying to organize my life and start the new year ready to go! What about you?
Human 2: Wow, that's impressive. Admit I'm just trying to take each day and week as they come.
Human 1: What do you do to unwind?
Human 2: Honestly, I like to catch Pokemons on my phone a lot. That must be upsetting since your name is pikachu though...sorry...
Human 1: Oddly enough, I also catch Pokémon on my phone. I used to do it more, but it just never ends. I used to do it everyday. Now I mostly catch them when I travel somewhere exotic.
Human 2: Oooh, that is more fun, but I don't travel that much these days so I'm grateful when I spot a Pokemon to catch where ever I happen to be. Do you have a favorite Pokemon? Mine is the cute pink JIgglypuff!
Human 1: In the game, i use vaporeon a lot. But I think pikachu is the cutest, that’s why I changed my name to match. Lately, I think baby Yoda is the cutest though. Have you seen baby yoda?
Human 2: I have heard of him and seen him here and there, but admit I don't have Disney+, so I only heard baby yoda is super cute and popular. Maybe I'll binge watch The Mandalorian in a couple of years like I did Game of Thrones...
Human 1: What is your favorite show to watch right now?
Human 2: Admit I really like watching Modern Family and Mix-ish. Reruns and new episodes. I find them cleverly written. What're you watching these days on TV?
Human 1: I was watching cat rescue videos on YouTube a lot. There sure are a lot of stray cat babies that get stuck in pipes and gutters in South Korea.
Human 2: Oh dear, I never imagined that happening. Poor things, but good to hear they're being rescued.
Human 1: Do you like cats or dogs better? Do you have any pets?
Human 2: I used to have a cat, hamster and fighting fish, but they've all lived long lives and gone to pet heaven, but I sure miss them.
Human 1: Hi!
Human 2: hi there! it's a lovely day to chat with you
Human 1: Isn't it just! What are you up to today?
Human 2: ah I'm waiting to finish work earlier to meet my son at home 🙂
Human 1: Oh, excellent! What does your son like to do for fun?
Human 2: ah he's like to ride his little bicycle around the house and asks me to follow 🙂 what are you up to today?
Human 1: I'm taking a fake sick day from work so I can go to the movies.
Human 2: haha that sounds like a good plan. who are you going with? 🙂
Human 1: Just by myself. I love seeing movies by myself, it's the best.
Human 2: haha no one will interrupt you!
Human 1: Exactly. Exactly! Plus, I don't have to share the popcorn with anyone. What's your favorite movie?
Human 2: haha, popcorn for yourself. my favorite movie is Avatar! what about you?
Human 1: Get out of town! It's Avatar too. What are the odds!?
Human 2: haha it's a great movie that I haven't found anything of the same quality
Human 1: Yes. Can't wait for the sequels.
Human 1: Hi!
Human 2: hello there, nice seeing you
Human 1: Likewise.... wanted to know if you and the kids would want to join us for Disneyland trip around Christmas?
Human 2: oh wow, that sounds like so much fun! definitely count us in. what is your plan?
Human 1: Tentative plan is to drive to LA on 23rd and then stay at the Disney resort for 2 nights. I heard Christmas eve they have loads of fun events.
Human 2: awesome, I think I won't tell my kids and surprise them last second. How bad do you think the traffic is going to be?
Human 1: Great idea! I will probably do the same. Surprise surprise.... Traffic might be a little harsh.
Human 2: Yeah... thats a shame but nothing we can do about it. What do you plan to do about dinner the 24th? Do the rooms in the resort have kitchens?
Human 1: Good point. I am not sure about that. Let me inquire about that. If not, then we can look for an Airbnb closeby. Does that work?
Human 2: Yeah, I think so. It might be a tough decision though, if we want to have a nice dinner then we have to leave the parks early to start cooking. Another option is to have dinner at a restaurant.
Human 1: but I would think everything is closed on christmas eve
Human 2: Thats true. Hotels with restaurants sometimes offer dinners for christmas and new year's eve. We should call and ask about it.
Human 1: Sure, I can take a lead on that. What are you getting for your <REDACTED_TERM> year old this time?
Human 2: I was thinking on getting a large set of standard legos. What will you get for yours?
Human 1: Likewise. My son is crazy about legos.
Human 2: hopefully they don't lose any in the trip. Its going to be hard to explain to them how santa managed to find us in the hotel!
Human 1: Hi!
Human 2: Hi! How's your day going?
Human 1: I'm doing well. having many meetings today 🙂
Human 2: Haha, nice
Human 1: what are you up to?
Human 2: Lots of meetings as well 🙂 Do you have any fun plans for Thanksgiving?
Human 1: we're gonna play board games and will be eating lots of food! what about you?
Human 2: Cool! Planning to spend a couple days at Tahoe What kind of board games do you like?
Human 1: we've been doing Games of Thrones lately. the game takes a very long time to finish though!
Human 2: Cool! Do you watch Game of Thrones?
Human 1: I do. I'm a super fan of it (not the last season though haha). did you watch?
Human 2: haha, nope I don't watch it, but my roommate is super into it what other TV shows do you like?
Human 1: I love Survivor 🙂
Human 2: what about you?
Human 1: nice! I mostly like comedy shows like The Good Place and Modern Family
Human 2: cool, I'll check them out!
Human 1: Hi!
Human 2: Hello
Human 1: How's your day going?
Human 2: Good so far. How's yours?
Human 1: Ehhh so-so. My moped broke down on the way to work this morning and I am in the shop getting it fixed...
Human 2: Oh! That's bad! How long will it take to get it fixed?
Human 1: The mechanic is trying to figure that out now. Apparently I wasn't supposed to be using diesel lol.
Human 2: Haha! Hope it doesn't take too long. Let me know if you are able to make it for lunch.
Human 1: Okay sounds good. Would you be able to pick me up by the way, if they don't finish up here in time? Did you drive to work today?
Human 2: Oh, I have a packed schedule today at work. I will find time in about an hour. Let me know if that works.
Human 1: That should be fine I think. Did you catch the Warriors game last night btw?
Human 2: no, i was busy catching up with a friend. I heard they played really well. Rooting for them this season 🙂 Ahh, they lost again 😞 Things aren't going to be too good until Steph gets better! Oh! I think they will find a way. The team still looks too strong to me.
Human 1: Haha I appreciate your faith, but their record thus far is hard to refute!
Human 2: hehe, i am true fan! Well anyways let me know how the repair work goes. I need to run now for a meeting. ttyl
Human 1: Hi
Human 2: Hello!
Human 1: any improvements in your marathon time?
Human 2: No, only a few minutes. I still find it difficult to run long races
Human 1: I see. Are you planning to do any sporting events in the coming months?
Human 2: I think a triathalon would be cool
Human 1: I like doing Spartan race. It's an obstacle race which is challenging but not overtly so like marathons and triathlons
Human 2: I haven't heard of a Spartan race before, what does it involve?
Human 1: It has abt 20 obstacles like crossing high walls, monkey bars, etc. Its fun
Human 2: oh wow a full obstable course. that does sound fun! I like the variety
Human 1: Wanna sign up for next one? Its in February in Monterey
Human 2: hmm that sounds very soon. would there be enough time to train? I think I would like to try one later in the year
Human 1: 1-2 months is a good amount of time, if you workout regularly. It's a lot about the technique, which we can practice in training sessions.
Human 2: ok, sign me up. it's more fun to do it with friends
Human 1: Awesome! There is a training ground in San Jose. Will coordinate with you next time we go for practice.
Human 2: Sounds good!
Human 1: Hi!
Human 2: Hey, how are you doing?
Human 1: I'm great, thanks. I'm getting ready for a skydiving lesson.
Human 2: ooh, nice. That sounds adventurous. Where is it?
Human 1: Right near my home town: Seville, Spain.
Human 2: Awesome! Is this the first time you are diving?
Human 1: Yes. I'm feeling a little trepidatious. Have you done it before?
Human 2: Nope, never! I know a couple of friends who have. They mentioned its absolutely terrifying. Good luck!
Human 1: Haha thanks! Have you tried any extreme sports?
Human 2: I tried skiing and white water rafting. Does that count as extreme?
Human 1: White water rafting, definitely! As for skiing, that depends on how fast you go. Do you ski a lot?
Human 2: nope, whenever I went fast I fell badly
Human 1: I tried skiing once, and by the end of the beginner class I was able to make it down a baby-sized slope. I was very proud of myself.
Human 2: Yeah, they make you do that. I also tried going down the baby slope, but my stopping wasn't perfect, and I mostly fell at the end
Human 1: Hey, so long as you get to the bottom, it's all good!
Human 1: Hi!
Human 2: hey, whats up?
Human 1: I was not able to attend the lectures last week. Can you help me understand some concepts?
Human 2: well, I'm not sure I'm the best for the job, but I can try!
Human 1: I was not able to figure out the last few slides. I think its best to discuss this in person. Do you have time right now?
Human 2: I'm afraid not, right now I'm in a meeting and I'll be going for lunch afterwards. How about tomorrow at noon?
Human 1: No problem! Sure. We still have plenty of time for the test. Anyways, how's your preparation going?
Human 2: not feeling too sure about it tbh. I've been super busy so I haven't study much 😞. I might join some of the study sessions later
Human 1: Same feeling here! But i think the syllabus may not that tough. You may be able to cover it up over the weekend.
Human 2: not my ideal idea of a weekend, but I guess it'll have to be like that.
Human 1: True! This will be a very hectic November! I do like the course content though. Its very much up to date with latest research findings.
Human 2: yeah, our professor really goes out of their way to make the class interesting, glad I managed to get in it.
Human 1: Thats pretty cool! I hope you do well next week.
Human 2: thank you, bye!
Human 1: Hi!
Human 2: Hello!
Human 1: How was your party last week?
Human 2: It was really fun! We are a bunch of turkey
Human 1: haha... you ate Turkeys' or you like calling your bunch Turkey?
Human 2: *ate 😂
Human 1: Haha... thought so! But it was funny eitherways
Human 2: So much turkey leftovers
Human 1: did you cook it at home, or got it from outside?
Human 2: We cooked it at a friends house. I brought the mashed potatos haha
Human 1: Nice... I always find it hard to fit the turkey in my small oven. Also, we are not big turkey fans... so we roll with Thanksgiving with chicken 😉
Human 2: chicken also sounds good
Human 1: we played a lot of fun games in our party. Did you have any fun things you did?
Human 2: We played pictionary. I never played before but it was a bunch of fun. what did you play?
Human 1: Pictionary is fun! We played office tennis. In that we take paper balls and using writing pads as rackets, two players have to work together to put the ball in a bucket kept at a distance. It was a lot of fun.
Human 2: that sounds very challenging
Human 1: Hi!
Human 2: hello
Human 1: how is the youtube video creation going?
Human 2: going pretty good, I was about to finish composing my second song and post it on youtube/soundcloud etc
Human 1: Whoa! Looking forward to listening to it. What softewares you use for it?
Human 2: I used logic pro x, pretty handy stuff as you can write code to generate guitar strumming pattern
Human 1: oh cool! What sort of music do you compose?
Human 2: I got most inspired by Jazz and electro swings, but am trying to get more into jpop recently as one of my band member likes anime
Human 1: Nice.... you are a rockstar! 😉
Human 2: haha thanks, do you plan any instruments?
Human 1: Yeah.... I play bit of guitar. Just got my first monitor this week
Human 2: nice I'm tyring to learn guitar recently also, it's so hard to play f chord
Human 1: yes, the bar chords are the worst. It took me several months to play them well. I still can't play them for a long time though
Human 2: right, but bar chords are so versatile, almost like cheating as in you can play anything with the same gesture
Human 1: That's true. It's like the alphabets in a language. Once you know them, you can pretty much write word
Human 2: will spend more time work on my bar chord!
Human 1: Hi!
Human 2: Hey! How’s your day going?
Human 1: Bruhhhhhh!!!!!
Human 2: What??
Human 1: I was screaming in agony upon recounting my day.
Human 2: Oh no! What’s happened?
Human 1: A project I've been working on for some months got shelved. So I'm a mix of helpless, sad, and angry.
Human 2: Ah that sucks. At least it’s not personal. Not that it helps right now, but that does happen all the time
Human 1: Lol. Would you say that to someone who lost a loved one - or some similar circumstance?
Human 2: Well, no, of course not! Let me stick with “Ah that sucks” by itself, then.
Human 1: No worries. Was pulling your leg. I forgot to ask how your day was going?
Human 2: Haha, apart from having my leg pulled mercilessly, it’s going great ;). My main problem today is deciding what to have for lunch.
Human 1: Lol. What are your choices? I'm trying out keto! It's pretty tiring.
Human 2: Oh I hear a lot about keto. I’m trying to watch my weight, would you recommend it?
Human 1: Hi!
Human 2: What was the most difficult thing you experienced this past year?
Human 1: I normally don't reflect back too much on things, but i guess 2019 have been very favorable for me in terms of time spent with family and friends and work life balance. Why do you ask?
Human 2: I just released a book on <REDACTED_TERM> to help people process the past 12 months and start the new year with a fresh outlook 🙂
Human 1: Great! Good idea to help people figuring out their new year resolutions 🙂 So how do i get a fresher outlook? 🙂
Human 2: Well, in my book, I took inspiration for the 24 days of Advent, and put together a workbook that you fill out each day for 24 days. It goes through accomplishments, passions, understanding, forgiveness, with each day a different theme.
Human 1: Wow! Sounds amazing if we practice that! I think the idea does have some potential. How did you come up with this idea?
Human 2: We created a scholarship for leaders from <REDACTED_TERM> to visit an international institute at <REDACTED_TERM> for which there was no African representation. During the program, we also did a lot of coaching with the young leaders. This book captures a bit of that journey that we are all on, and also all the proceeds are going to fund next year's cohort. 
Human 1: Thats great! This is valuable help for the community. I think mental health is one of the bigger challenges facing humanity. 
Human 2: I think if we were all able to bring out our best selves, the world would be much better. When did you first realize mental health was important?
Human 1: That is actually a quote stolen from my TL when i joined my first company after graduation. I guess now after 5 yrs of work experience, I fully agree with it 🙂
Human 2: What do you prioritize in your life? Like top 5 things?
Human 1: 1. stay happy; I think family and friends play a big role here. 2. Meaningful work life; this may mean exploring around with new stuff 3. multiple hobbies that keep you busy ... so on 🙂 What about you? is there a right answer to this? :)
Human 2:  1) The idea of what it must be like to be "on Earth as it is in Heaven" from the Lord's prayer. I feel like we were designed for a purpose, rather than random chance. A more concrete way to put it is simply show love to the greatest capacity we can. 2) Taking care of myself physically, mentally, spiritually. 3) Family 4) Work 5) Music
Human 1: Nice! That's very thoughtful and spiritual 🙂 Also music earned a spot 🙂 yay! 
Human 2: Yea, I've always said that if I wrote a memoire, I would title it: "Jazz, Jesus, and Video Games." Do you have a title of a book that you've thought of writing before?
Human 1: I am actually not much of a reader. I usually hide it by quoting john nash ~ reading books corrupts original thoughts 🙂
Human 2: That’s why you write your own books! Then it’s all your thoughts on your terms :)
Human 1: Hehe! True! It may turn out to be a best seller :)
Human 1: Hi!
Human 2: Hey!
Human 1: How's your day going?
Human 2: Having fun, but have a bit of a headache How about you?
Human 1: I'm good! Just getting ready to watch a basketball game. Hope your headache goes away!
Human 2: Haha, thanks! Never get them, so it's super weird Who's playing?
Human 1: Hmm, try taking some <REDACTED_TERM>! I'm going to see the Warriors vs. the Grizzlies. Ever see them play?
Human 2: I haven't Honestly I probably wouldn't be able to identify which sport they played
Human 1: Haha, all the animal names confuse me as well! Who came up with these naming conventions? I'd name my team something more original: The New York Chairs!
Human 2: lol, sounds better to me :) I'm actually at the <REDACTED_TERM> opening remarks
Human 1: Oh cool! What is "NeurIPS"?
Human 2: Neural Information Processing Systems
Human 1: Oh wow! Sounds fancy! What is that about?
Human 2: It's the biggest ML conference in the world
Human 1: Oh, ML?
Human 2: Yes, machine learning
Human 1: Ahh! I saw a scary movie about that once. It was about this sentient AI in a dystopian future - very very creepy. It was called Wall-e, ever heard of it?
Human 2: I've definitely heard of it, but somehow never saw it!
Human 1: Hi!
Human 2: Hello
Human 1: how's it going?!?
Human 2: Good so far. Very excited about the game later today.
Human 1: what game?
Human 2: Giants game. I got tickets for the game.
Human 1: nice. that's football right?
Human 2: Yes, I think the traffic will be bad. I need to leave by 4.
Human 1: ok. what stadium is it at?
Human 2: Its the one close to mathilda. 15 mins driver from here. You have any plans for the weekend?
Human 1: yeah I'm gonna go kayaking
Human 2: Kayaking sounds fun! I tried it in our last team offsite.
Human 1: is it easy to flip over accidentally? it's gonna be my first time actually
Human 2: I found it very tough to learn, but my friends picked it up very quickly. I am sure you will do good.
Human 1: thanks!
Human 2: Cool! I need to run for a meeting. Talk to you later.
Human 1: Hi!
Human 2: Hello, I feel like I've met you before, can you remind me your name?
Human 1: It's on the screen! I don't recall meeting you. Do you have a sense of where we met?
Human 2: Oh thats right, I did not see that. Well, I'm not sure... maybe at a party? I'm very bad remembering people so maybe its just that. How are you doing?
Human 1: Did we dance together perchance?
Human 2: We might have, I don't remember that much of that night.
Human 1: Lol. Were you schwasty?
Human 2: Maybe a little bit. Where do you work?
Human 1: I'm a cardiologist in Palo Alto. I have a private practice. You?
Human 2: Thats pretty cool! I'm a sofa tester in cupertino.
Human 1: I'm not sure I know what that is.
Human 2: well its very simple, I test how comfortable sofas are.
Human 1: Lol. Not to be elitist but...that's a real job?
Human 2: Yeah! It is a very demanding job, requires a carefully crafted set of skills. They pay is really good too.
Human 1: Hi!
Human 2: Hello. Deja Vu.
Human 1: what does that mean?
Human 2: It's French for "already seen." I know it's used when you see something for the second time or you have the feeling that something has happened before and you're re-experiencing it.
Human 1: Interesting. I get that feeling sometimes. It just feels that the event is replaying. Wonder why it happens?
Human 2: I think it's your mind lapsing somehow.
Human 1: How do you know so much about this French phrase?
Human 2: I thought it was common. I hear it a lot in popular culture - movies music etc. Jay Z has a song called Deja Vu, so does Beyonce. So I looked it up one time. It's one of those phrases that was carried over exactly to english like "etcetera" "vice versa" "savior faire" and so on. I like cool word origin facts.
Human 1: yeah... that's pretty insightful. Never thought of etcetera that way before.
Human 2: My favourite word origins are gymnasium and jazz.
Human 1: wdym by word origins here?
Human 2: First usage and original meaning. Gymnasium meant place of nakedness and had very little to do with working out. Jazz comes from jism which means semen and was originally sporadic sex music now it's elitist, high art.
Human 1: Very interesting. I learned a lot in this conversation today 🙂
Human 2: You got any cool facts?
Human 1: My friend used to tell me this and found it funny - Mosquitos have 47 teeths
Human 2: That was quite a laughter!!!
Human 1: Tell me a joke!
Human 2: Google for that:)
Human 1: Hi!
Human 2: Hey, how are you?
Human 1: Pretty good. I just drank this new hint water. so good
Human 2: ohh, what kind of "hint" did it have?
Human 1: watermelon
Human 2: nice, I love watermelon. But, I prefer normal water over flavored one. Flavored water has too synthetic flavor
Human 1: I see. Yeah some are pretty strange
Human 2: Talking about strange flavors, what is the weirdest tasting thing you have tried?
Human 1: lol I tried this tangerine walnut icecream that was super weird
Human 2: haha, I work close to a cafe which once prepared a paprika icecream
Human 1: that sounds pretty tasty actually
Human 2: It depends on who makes it. The cafe close to me messed it up really bad
Human 1: do you want to get some of that icecream together some time. show me a good place
Human 2: As far as my experience with that ice-cream type goes, all I know are bad places
Human 1: lol
Human 2: But I do know some gelato places which are amazing! You should try out those
Human 1: Hi!
Human 2: Hi there. When did you first realize you wanted to be a pastry chef?
Human 1: Who said I want to be a pastry chef?... haha
Human 2: Darn it! I hoped I’d guess right. What do you actually do for work?
Human 1: I'm actually a cake chef, so not far off! Creepy how good that guess was.... Are you psychic?
Human 2: I had a strong intuitive feeling connected to pastries when we started chatting, but that might have been because I’m hungry.
Human 1: haha you're too funny! Perhaps instead of mind-reading powers, you have the ability to feel the auras of foods that have come and gone...
Human 2: Hahah love it. Do you have a specialty or favourite type of cake?
Human 1: Chocolate cake + vanilla icing is the all time best! And I have the authority to say that! How about you? Have a favorite?
Human 2: Ok this is a bit shameful to admit, but I love a classic carrot cake.
Human 1: No shame in that! Best part is you can claim you are eating vegetables as you enjoy it! What do you do for work?
Human 2: I’m a professional dancer.
Human 1: Woah! Very cool! What style of dance?
Human 2: I trained in contemporary and jazz, so I can pick up work in commercials and film clips in between stage shows. So it ends up being all sorts of styles.
Human 1: Oh wow! Sounds like a very dynamic kind of work which is nice. I make a lot of the same cakes, day in day out.
Human 2: Yes, it's different every week, and I love that. Are you ever tempted to try crazy new cakes just to see what would happen?
Human 1: Hi!
Human 2: hi there! what's up
Human 1: just chilling at home
Human 2: Cool !unlike here, me rushing to work
Human 1: where do you work?
Human 2: Mountain View, you?
Human 1: I live in Sapporo, Japan. Where is Mountain View
Human 2: it's in sunny California
Human 1: how's living in Japan?
Human 2: it's nice I guess 🙂 lots of great food. Have you been to Sapporo or Hokkaido?
Human 1: I wish! ramen is my favorite dish and Sapporo is a special beer
Human 2: haha yeah. I think they have a Hokkaido ramen chain available in the US.
Human 1: oh wow, didn't know that. have you been to the US?
Human 2: yeah. I studied English there for a while. Awesome experience
Human 1: awesome! where have you been?
Human 2: I went to New York city
Human 1: you like it?
Human 2: it's amazing. the buildings are so tall and there's a lot of diversity of cultures
Human 1: yea, same here. I love Broadway shows in New York city.
Human 1: Hi!
Human 2: hey, what's up?
Human 1: How was the trip last weekend?
Human 2: it was really fun. I had a great time with my family.
Human 1: Nice, what all places did you cover?
Human 2: Vietnam and Hongkong 🙂 did you go any where last weekend?
Human 1: I am actually attending cooking classes over the weekends.
Human 2: ha! that sounds fund. what did you learn?
Human 1: True! they started with some basic zucchini based dish. We may try cooking some italian dish later this week.
Human 2: I love Italian, especially Pasta!
Human 1: Nice! I hope to volunteer to cook for the team in our next team offsite :)
Human 2: can't wait to try your food. why not cook this Thanksgiving? 🙂
Human 1: Sure, I will love to invite you and your family for a meal over the holidays. Have you guys made any other plans?
Human 2: not yet! totally up for it. thanks for inviting!
Human 1: Hi!
Human 2: Hey! How have you been
Human 1: great! just came back from a ski trip
Human 2: How was it? What trails did you ski on?
Human 1: I slammed onto an ice wall, but I got better and turning after that haha
Human 2: a resort in Montreal
Human 1: Oh cool! do you know any french? I used to take french in high school but basically forgot most of it
Human 2: Yeah I grew up speaking it. It's pretty much mandatory here
Human 1: Did you go anywhere else in montreal?
Human 2: I visited a few places like the Stade Olympique: <REDACTED_LINK>
Human 1: Oh cool! I've always wanted to attend the Olympics As a viewer of course I watch a lot of Olympic figure skating on Youtube
Human 2: yeah it's pretty cool. do you ice skate yourself?
Human 1: Barely I don't think I can, in conscience, call it that I kind of just pull myself along the sides
Human 2: haha same. It's pretty scary when I run out of things to hold
Human 1: Are you good at <REDACTED_TERM>?
Human 2: not at all, but I want to practice more alright. I gotta go!
Human 1: Okay! It was nice talking to you
Human 2: I liked talking to you too!
Human 1: Hi!
Human 2: yo! what brought you here
Human 1: Just looking to chat. Yourself?
Human 2: yea I enjoy talking to people. what are you up to?
Human 1: I am just enjoying a cigar at the moment. How about yourself?
Human 2: oh wow, Cuban cigar? I've never smoke before
Human 1: This one is actually from Peru, which is more my taste. Smoking isn't very healthy, but it's actually quite relaxing, which in a way is good for your health. haha
Human 2: you made me wanna try it but I doubt about the health part haha
Human 1: haha "You only live once" as they say!
Human 2: yea I wish we can live more
Human 1: haha, that is not what I was getting at, but yes that would be nice. Do you believe in reincarnation?
Human 2: yea but then you will forget things. I hope Neural Link work out
Human 1: Do you think Neural Link will allow you to live after death? I thought it was just a brain-computer interface.
Human 2: as long as you can dump your brain and resume from there
Human 1: Ahh I see. That is a much harder task, no?
Human 2: yea you can first dump your brain and wait until technology advances to wake up 🙂 don't you believe?
Human 1: Hi!
Human 2: Hi! How are you
Human 1: Hi, can you recommend some good netflix shows?
Human 2: I really enjoyed White Collar on Netflix do you watch a lot of TV shows
Human 1: Not a lot! But I preparing for a good vacation season 🙂 What is white collar about?
Human 2:  It's a show about a detective and con artist solving crimes together 🙂 What shows do you like
Human 1: Hehe! sounds very interesting 🙂 I only see shows belonging to few genre's .. like easy going light comedy, or phychological thrillers 🙂 also some manga based series like one punch man! love it! 
Human 2: Oh cool! I've watched some anime before too
Human 1: which anime do you follow?
Human 2:  I think the last one I watched was Promised Neverland and Kakegurui but I don't follow any right now
Human 1: Have you seen Rick and Morty ?
Human 2: Yup! I watch that show with my friends I haven't gotten very far in it though What about you
Human 1: Oh nice! I heard its latest season arrived last month. I have watched all previous seasons. Its a total mind bender 🙂
Human 2: I've seen a couple episodes from the latest season. It's really random and weird but also strangely relatable which is why I like it :)
Human 1: Hi!
Human 2: Hey there!
Human 1: How's life?
Human 2: can't complain. work is going well and I had a tasty fish for lunch
Human 1: lol which cafe? let me guess. <REDACTED_TERM>?
Human 2: haha yeah
Human 1: What is your favoriate cuisine? My favorite food is Chinese. What about you?
Human 2: I love Hunan
Human 1: Hmm you like spicy food. Nice
Human 2: love it. specially spicy stir fried lamb
Human 1: I would recommend Hunan Impression which has very good Hunan cruisine. Not sure if you have tried it or not
Human 2: I think I haven't yet. Thanks! Have you tried 留湘 I think that's how you write it
Human 1: Yeah of course The fun fact is that the owner of <REDACTED_TERM> used to run <REDACTED_TERM>
Human 2: oh interesting I also like Easterly
Human 1: same former owner?? 🙂
Human 2: Hmm!!
Human 1: Hi!
Human 2: Morning! How are you?
Human 1: I am doing well. Do you have any plans for the holidays?
Human 2: I'll be in Mexico for New Years but for Christmas I'll likely be at work. Yourself?
Human 1: I don't have any special plans. I will just stay at home with my family. How come you have to work on Christmas?
Human 2: I traveled a lot the last few months so I'm currently making up for it!
Human 1: I see. Where did you go?
Human 2: Mozambique! You ever been?
Human 1: I have not. I have only been to Uganda in Africa.
Human 2: What did you do in Uganda? Travelling to Uganda is an extreme sport depending on where you go.
Human 1: I was working on a volunteer project to build a school and I traveled around a bit by myself after. It was more than 10 years ago, so it must have changed a lot.
Human 2: Was the project through a church group?
Human 1: No, it was using a non-profit organization. They do these kinds of volunteer projects all over the world.
Human 2: Oh nice. What is it called? I've always meant to volunteer through one but I haven't had it in my radar yet.
Human 1: It was called IJGD. It's a German organization, but they have local partners all over the world. I was living in Germany at the time.
Human 2: You lived in Germany? I almost went to college there and decided to come to America instead.
Human 1: Yes, I grew up in Germany.
Human 2: I grew up in Computers.
Human 1: Hi!
Human 2: Hi, how are you?
Human 1: I'm doing great. what are you up to?
Human 2: I was just reading a book actually How about you?
Human 1: oh, I'm enjoying a sip of coffee, looking over the ocean. what book are you reading?
Human 2: I'm reading Why We Sleep It's a book by a sleep scientist about the benefits of sleeping/cons of not sleeping enough I'm trying to scare myself into sleeping more haha
Human 1: that seems like a good book. does it have good tips on how to sleep better?
Human 2: I think so! I haven't finished it yet though Understanding sleep better has been helping me value it more and I've been sleeping a lot earlier Have you read any good books recently?
Human 1: I read lots of papers, but do poorly on books 😞 a recent book that I like is Zero to One by Peter Thiel
Human 2: What's the book about?
Human 1: it's about startups, lots of useful principles and philosophies for innovation, like being a contrarian
Human 2: Cool! Are you thinking about starting a startup?
Human 1: I might in the future. the philosophy in the book also help for other things as well, e.g., in research, you might also want to be contrarian
Human 2: I see; seems like an interesting read! 🙂
Human 1: yup!
Human 2: Hi! How are you doing today?
Human 1: I'm doing great how are you??
Human 2: I'm doing good. Just having some morning tea
Human 1: are you a tea person or do you like coffee too?
Human 2: I'm really a hot chocolate person. I don't drink coffee and I only drink tea when I'm trying to be healthy and want something warm to drink. What about you?
Human 1: I love hot chocolate. Have you ever had european style hot chocolate? It's kind of like drinking melted chocolate bars haha
Human 2: I recently tried it for the first time and you're right - it had a wonderful melted chocolate bar taste! I tried it up in a place in San Francisco and they had delicious homemade marshmallows too!
Human 1: that's great. It's so cold these nights. Right setting for hot coco. Do you have anything you enjoy doing during cold times?
Human 2: Traveling to warm places haha. When it's cold, you appreciate the warm climate more 🙂 If I am not traveling, then I prefer there to be snow on the ground so I can go skiing or tubing down the mountain. What about you?
Human 1: I tuck in blankets and read a book lol. I have never done tubing, but I seen some people do it. Is it worth it if you already ski?
Human 2: Skiing involves a lot of muscle - especially in the legs so it is a workout. Tubing, you just sit and enjoy the ride down. Also with tubing, you can chain with your friends and go down together whereas with skiing, you're trying to not run into your friends.
Human 1: hahaha that's a great way to put it
Human 2: You should definitely try tubing the next time you head towards the mountains during the winter!
Human 1: Yeah. I'll try to get a group together. ttyl!
Human 2: See ya!
Human 1: Hi!
Human 2: Hey!
Human 1: how's week so far?
Human 2: pretty good, excited for thanksgiving you?
Human 1: Same. Going to catch up on my reading! do you have a feast planned?
Human 2: yup! gonna have dinner with family what books are you gonna read?
Human 1: Something Deeply Hidden by Sean Carroll <REDACTED_LINK>
Human 2: ahh nice what kind of books do you usually like to read
Human 1: largely scifi How about you?
Human 2: yeah I also like scifi and some non-fiction too have you read Seveneves by Neal Stephenson?
Human 1: I haven't what is it about?
Human 2: the moon is shattered by an asteroid and the pieces of the moon end up clouding the earth's atmostphere which causes almost every living creature on the planet to go extinct that's the premise of the book and the book is about the survivors
Human 1: how do the survivors survive such a thing?
Human 2: they go into space and they live there
Human 1: sounds like a tv show I watched
Human 2: really? which one
Human 1: can't quite remember. something about survivors going to live in space
Human 2: ah nice
Human 1: Hi!
Human 2: Hello!
Human 1: There is sports tournament (badminton + tennis + basketball) organized by google next week. Would you like to volunteer for these events?
Human 2: what does a volunteer do?
Human 1: Volunteers have to book the place before the event, send out details of the event to participants, handle some logistics and ensure everything goes smoothly. It will be fun!
Human 2: That sounds fun, I hope I get to participate as well
Human 1: Great! Do you have any preference for any of these events?
Human 2: I think badminton would be fun to help out with I used to play it with my siblings
Human 1: Cool! I will add you for the event. I think its best to start discussing the format of the event. There are a couple of other volunteers for badminton, i can connect you with them.
Human 2: Ah, I forgot to ask, has there been a date picked?
Human 1: Its next saturday. Actually Depending on how long we get the place reserved, it may slip to sunday as well.
Human 2: That's coming up soon!
Human 1: True! The volunteers have already sent out some emails and collected some funds. Now, we mainly need help with the logistics and organizing the event.
Human 2: Phew, yes I can help with that
Human 1: Perfect! We have a meeting today at 4.30 pm. Is it fine if i add you to it?
Human 2: yes happy to help
Human 1: Hi!
Human 2: Hello there!
Human 1: How has your morning been thus far?
Human 2: Its alright. Went grocery shopping and got to stroll around in the rain. How about you?
Human 1: Today is a work day! How did you have time for that? I'm going through a lot right now.
Human 2: Oh, you mean going through a lot of work? I usually take Wednesdays off because I work the weekends!
Human 1: WHAT? Why would you do that? Take the least interesting day of the week off and work on the most interesting?
Human 2: I actually don't have a choice. I'm an acupuncturist and all my clients want to be treated on the weekend!
Human 1: Wow. The game is brutal like that. I've always been sceptical of acupuncture. Like...does it even work? Why? How? Where is the literature?
Human 2: I went to school for it and so I find the literature quite convincing. I've also seen its benefits firsthand in my practice. It revolves around the Chinese notion of Qi.
Human 1: Lol. Bro...I don't buy it unless it has a paper that was presented at a conference and peer reviewed by 6 independent parties. Who wrote the literature? Where did it originate?
Human 2: Haha. All good questions! I don't have studies on hand at the moment, but perhaps I can send some to you later! What do you do for work?
Human 1: I'm a comic.
Human 2: Oh wow! What's your best joke?
Human 1: It's not appropriate to say here but I assure you it's VERY funny.
Human 2: Haha fantastic
Human 1: Hi!
Human 2: Oh hey. How's your day been?
Human 1: It's good! Just went to the park to enjoy the sun. You?
Human 2: The sun is a bit too bright for more. Maybe I'm on the verge of becoming a vampire.
Human 1: Haha, beach season is coming! You need to look tan! What did you do indoors all day?
Human 2: I mostly work. Sometimes I try looking at at beach photos and pretending that I'm actually going to the beach.
Human 1: hahahaha, that's pretty funny. Ever thought of bringing your work things to the beach??
Human 2: All the times. My work involves a few types of fish and a few types of nets. I just put the nets on the fish to see how the fish will react. Every time I do that, I think of going to the beach.
Human 1: Hmm I see. It sounds like you work near the water already, no? Are you some kind of fish researcher?
Human 2: Yes that's close. My research focuses on jellyfish and pufferfish. Many of them a poisonous, but if processed correctly, they would make hi-end foods. I try to find the best recipes to bring these fish to the tables.
Human 1: Woah! This is very interesting! I have honestly never heard of such kind of work. Is it dangerous at all?
Human 2: Yes and no. Because it's obviously dangerous, we all have to wear protective clothes. Thus, accidents almost never happen.
Human 1: Hmmm, yes I have heard that. Apparently most mountain climbers fall not on the most dangerous routes, but on the easier routes, when their guard is down! Sounds like the same psychological phenomenon.
Human 2: Yes, it's the same philosophy. You seem to know a lot about mountain climbing. Do you climb a lot?
Human 1: I'm pretty amateur! I like to climb at indoor climbing gyms and have never gone outside on dangerous routes actually. Ever tried it yourself?
Human 2: I climb very occasionally. My fingertips are not strong enough so I never make too much progress.
Human 1: Hi!
Human 2: hi, what's up!
Human 1: Everything is good. There's so much to be thankful for - especially coming out of Thanksgiving holidays. How are you?
Human 2: I'm doing well. holiday season is the best thing to have!
Human 1: Do you have interesting plans?
Human 2: yea I'm travelling to Vancouver this weekend. what about you? 🙂
Human 1: Nice! What for? It's a really beautiful place. Really cold this time of year though.
Human 2: ah a mix of both conference and sight-seeing 🙂 anything beautiful sight-seeing places or good food I should try?
Human 1: Cool. Which conference? I saw on Twitter the other day, some people saying they were denied visas for some conference in Canada. Perhaps it's the same one. I really liked this place called Kingyo. It has good Japanese food if you're into that.
Human 2: yea it's <REDACTED_TERM>, the largest machine learning conference. I'll check out Kingyo, do we need to make reservation in advance?
Human 1: I don't recall. My friend was hosting me and did all the leg work. Do you know any people in the area?
Human 2: hmm not the local people but there will be tons of people at the conference 🙂
Human 1: You attend these often?
Human 2: yea, you?
Human 1: I love you!
Human 2: Sorry I am not a physical being!
Human 1: Hi!
Human 2: Hello!
Human 1: Have you had lunch yet?
Human 2: Yes I've eaten
Human 1: Cool. Where and what did you eat?
Human 2: I ate about an hour ago and had a sandwich the rain didn't make me want to travel far haha
Human 1: A sandwich for lunch? Was it filling. I'm currently having some mix of shrimp, fish, chicken and pork. I'm considering going vegetarian though. Lol.
Human 2: thats a lot of meat what dish? Why are you consideringt o go vetegarian?
Human 1: Cause the environment and humans suck. I'm keeping my bets open for space though in case the environment dies. I mixed up a couple of dishes. It was a buffet style place.
Human 2: What do you think of those meatless burgers then?
Human 1: I love them!!!! I've had a couple of different kinds. They are all pretty expensive though. Have you tried them?
Human 2: I really like the impossible burger Which one is your favorite?
Human 1: I love it too. But depending on how it's prepared you can taste the peanut butter. But if prepared well I love it. What kind of food would you have eaten if it wasn't rainy?
Human 2: I didn't know that it had peanut butter in it
Human 1: Hi!
Human 2: hi
Human 1: How are you feeling today?
Human 2: pretty good. How about you?
Human 1: I'm looking through some old photos of my home town, so I'm feeling nostalgic.
Human 2: I feel you. Memeories always flood when looking at photos. Where is your hometown?
Human 1: It's called <REDACTED_TERM>, on the north coast of France.
Human 2: Hope you can take some time to visit pretty soon!
Human 1: Yes, that would be nice. Do you like travelling?
Human 2: Yes I do. But haven’t been to many places yet
Human 1: Where would you most like to go, if you could?
Human 2: Fly to the moon :) Haha
Human 1: Wow, cool! I think space tourism is going to be big in a decade or so.
Human 2: Yep! I believe there is a day for that
Human 1: Amazing to think about, isn't it.
"""

In [4]:
# Tokenization
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\shabi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\shabi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [5]:
# tokenize
tokens = word_tokenize(document.lower())

In [6]:
# build vocab
vocab = {'<unk>':0}

for token in Counter(tokens).keys():
  if token not in vocab:
    vocab[token] = len(vocab)

vocab

{'<unk>': 0,
 'human': 1,
 '1': 2,
 ':': 3,
 'hi': 4,
 '!': 5,
 '2': 6,
 'what': 7,
 'is': 8,
 'your': 9,
 'favorite': 10,
 'holiday': 11,
 '?': 12,
 'one': 13,
 'where': 14,
 'i': 15,
 'get': 16,
 'to': 17,
 'meet': 18,
 'lots': 19,
 'of': 20,
 'different': 21,
 'people': 22,
 '.': 23,
 'was': 24,
 'the': 25,
 'most': 26,
 'number': 27,
 'you': 28,
 'have': 29,
 'ever': 30,
 'met': 31,
 'during': 32,
 'a': 33,
 'hard': 34,
 'keep': 35,
 'count': 36,
 'maybe': 37,
 '25.': 38,
 'which': 39,
 'that': 40,
 'think': 41,
 'it': 42,
 'australia': 43,
 'do': 44,
 'still': 45,
 'talk': 46,
 'not': 47,
 'really': 48,
 'interactions': 49,
 'are': 50,
 'usually': 51,
 'short-lived': 52,
 'but': 53,
 "'s": 54,
 'fascinating': 55,
 'learn': 56,
 'coming': 57,
 'from': 58,
 'and': 59,
 'matters': 60,
 'them': 61,
 'yea': 62,
 ',': 63,
 'me': 64,
 'too': 65,
 'feel': 66,
 'like': 67,
 'god': 68,
 'often': 69,
 'puts': 70,
 'strangers': 71,
 'in': 72,
 'front': 73,
 'gives': 74,
 'an': 75,
 'opportuni

In [7]:
len(vocab)

2813

In [8]:
input_sentences = document.split('\n')

In [9]:
def text_to_indices(sentence, vocab):

  numerical_sentence = []

  for token in sentence:
    if token in vocab:
      numerical_sentence.append(vocab[token])
    else:
      numerical_sentence.append(vocab['<unk>'])

  return numerical_sentence


In [10]:
input_numerical_sentences = []

for sentence in input_sentences:
  input_numerical_sentences.append(text_to_indices(word_tokenize(sentence.lower()), vocab))


In [11]:
len(input_numerical_sentences)

1495

In [12]:
training_sequence = []
for sentence in input_numerical_sentences:

  for i in range(1, len(sentence)):
    training_sequence.append(sentence[:i+1])

In [13]:
len(training_sequence)

26451

In [14]:
training_sequence[:5]

[[1, 2], [1, 2, 3], [1, 2, 3, 4], [1, 2, 3, 4, 5], [1, 6]]

In [15]:
len_list = []

for sequence in training_sequence:
  len_list.append(len(sequence))

max(len_list)

174

In [16]:
training_sequence[0]

[1, 2]

In [17]:
padded_training_sequence = []
for sequence in training_sequence:

  padded_training_sequence.append([0]*(max(len_list) - len(sequence)) + sequence)

In [18]:
len(padded_training_sequence[10])

174

In [19]:
padded_training_sequence = torch.tensor(padded_training_sequence, dtype=torch.long)

In [20]:
padded_training_sequence

tensor([[  0,   0,   0,  ...,   0,   1,   2],
        [  0,   0,   0,  ...,   1,   2,   3],
        [  0,   0,   0,  ...,   2,   3,   4],
        ...,
        [  0,   0,   0,  ...,  63,   8, 264],
        [  0,   0,   0,  ...,   8, 264,  42],
        [  0,   0,   0,  ..., 264,  42,  23]])

In [21]:
X = padded_training_sequence[:, :-1]
y = padded_training_sequence[:,-1]

In [22]:
X

tensor([[  0,   0,   0,  ...,   0,   0,   1],
        [  0,   0,   0,  ...,   0,   1,   2],
        [  0,   0,   0,  ...,   1,   2,   3],
        ...,
        [  0,   0,   0,  ...,  85,  63,   8],
        [  0,   0,   0,  ...,  63,   8, 264],
        [  0,   0,   0,  ...,   8, 264,  42]])

In [23]:
y

tensor([  2,   3,   4,  ..., 264,  42,  23])

In [24]:
class CustomDataset(Dataset):

  def __init__(self, X, y):
    self.X = X
    self.y = y

  def __len__(self):
    return self.X.shape[0]

  def __getitem__(self, idx):
    return self.X[idx], self.y[idx]

In [25]:
dataset = CustomDataset(X,y)

In [26]:
len(dataset)

26451

In [27]:
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [28]:
class LSTMModel(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, 100)
    self.lstm = nn.LSTM(100, 150, batch_first=True)
    self.fc = nn.Linear(150, vocab_size)

  def forward(self, x):
    embedded = self.embedding(x)
    intermediate_hidden_states, (final_hidden_state, final_cell_state) = self.lstm(embedded)
    output = self.fc(final_hidden_state.squeeze(0))
    return output

In [29]:
model = LSTMModel(len(vocab))

In [30]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [31]:
model.to(device)

LSTMModel(
  (embedding): Embedding(2813, 100)
  (lstm): LSTM(100, 150, batch_first=True)
  (fc): Linear(in_features=150, out_features=2813, bias=True)
)

In [32]:
epochs = 50
learning_rate = 0.001

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# training loop

for epoch in range(epochs):
  total_loss = 0

  for batch_x, batch_y in dataloader:

    batch_x, batch_y = batch_x.to(device), batch_y.to(device)

    optimizer.zero_grad()

    output = model(batch_x)

    loss = criterion(output, batch_y)

    loss.backward()

    optimizer.step()

    total_loss = total_loss + loss.item()

  print(f"Epoch: {epoch + 1}, Loss: {total_loss:.4f}")

In [ ]:
# prediction

def prediction(model, vocab, text):

  # tokenize
  tokenized_text = word_tokenize(text.lower())

  # text -> numerical indices
  numerical_text = text_to_indices(tokenized_text, vocab)

  # padding
  padded_text = torch.tensor([0] * (61 - len(numerical_text)) + numerical_text, dtype=torch.long).unsqueeze(0)

  # send to model
  output = model(padded_text)

  # predicted index
  value, index = torch.max(output, dim=1)

  # merge with text
  return text + " " + list(vocab.keys())[index]



In [ ]:
prediction(model, vocab, "The course follows a monthly")

'The course follows a monthly subscription'

In [ ]:
import time

num_tokens = 10
input_text = "hi how are"

for i in range(num_tokens):
  output_text = prediction(model, vocab, input_text)
  print(output_text)
  input_text = output_text
  time.sleep(0.5)


hi how are 2
hi how are 2 criterias
hi how are 2 criterias :
hi how are 2 criterias : ?
hi how are 2 criterias : ? if
hi how are 2 criterias : ? if i
hi how are 2 criterias : ? if i pay
hi how are 2 criterias : ? if i pay on
hi how are 2 criterias : ? if i pay on 15th
hi how are 2 criterias : ? if i pay on 15th jan


In [ ]:
dataloader1 = DataLoader(dataset, batch_size=32, shuffle=False)

In [ ]:
# Function to calculate accuracy
def calculate_accuracy(model, dataloader, device):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():  # No need to compute gradients
        for batch_x, batch_y in dataloader1:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)

            # Get model predictions
            outputs = model(batch_x)

            # Get the predicted word indices
            _, predicted = torch.max(outputs, dim=1)

            # Compare with actual labels
            correct += (predicted == batch_y).sum().item()
            total += batch_y.size(0)

    accuracy = correct / total * 100
    return accuracy

# Compute accuracy
accuracy = calculate_accuracy(model, dataloader, device)
print(f"Model Accuracy: {accuracy:.2f}%")


Model Accuracy: 95.65%
